# 🏛️ Arquitecturas Modernas de Datos: Lambda, Kappa, Delta y Data Mesh

Objetivo: comprender y contrastar arquitecturas de referencia (Lambda, Kappa, Delta) y patrones organizacionales (Data Mesh), con ejemplos de diseño y trade-offs.

- Duración: 120–150 min
- Dificultad: Alta
- Prerrequisitos: Mid completo, experiencia con batch y streaming

## 1. Arquitectura Lambda

### 🏗️ **Lambda Architecture: Batch + Speed Layer**

**Origen y Motivación (Nathan Marz, 2011):**

```
Problema clásico:
- Batch processing: Preciso pero lento (horas/días)
- Stream processing: Rápido pero complejo (estado, fallos)

Solución Lambda:
- Batch Layer: Verdad absoluta, inmutable, reprocessable
- Speed Layer: Aproximación rápida, eventualmente consistente
- Serving Layer: Merge de ambas vistas
```

**Arquitectura Completa:**

```python
┌─────────────────────────────────────────────────────────────┐
│                     DATA SOURCES                             │
│  • Kafka Topics                                              │
│  • Database CDC (Debezium)                                   │
│  • Application Logs                                          │
│  • IoT Streams                                               │
└─────────────────┬───────────────────────────────────────────┘
                  │
        ┌─────────┴──────────┐
        │                    │
        ▼                    ▼
┌──────────────┐    ┌──────────────┐
│ BATCH LAYER  │    │ SPEED LAYER  │
│              │    │              │
│ • Spark Batch│    │ • Flink      │
│ • Hadoop MR  │    │ • Spark SS   │
│ • Hive       │    │ • Storm      │
│              │    │              │
│ Runs: Daily  │    │ Runs: Real-  │
│       @2AM   │    │       time   │
└──────┬───────┘    └──────┬───────┘
       │                   │
       │ Master Dataset    │ Delta Views
       │ (Immutable)       │ (Mutable)
       │                   │
       ▼                   ▼
┌─────────────────────────────────┐
│      SERVING LAYER               │
│                                  │
│  Query(t) = Batch(0→t-1h) +     │
│             Speed(t-1h→t)        │
│                                  │
│  • Druid                         │
│  • Cassandra                     │
│  • ElasticSearch                 │
│  • BigQuery                      │
└─────────────┬───────────────────┘
              │
              ▼
        ┌──────────┐
        │   API    │
        │Dashboard │
        └──────────┘
```

**Componentes Detallados:**

**1. Batch Layer (Verdad Inmutable):**

```python
# Características:
# - Procesamiento completo del histórico
# - Recalculable desde el inicio
# - Optimizado para throughput (no latencia)
# - Tolerancia a fallos simple (restart)

# Ejemplo: Agregaciones diarias con Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, window

spark = SparkSession.builder \
    .appName("BatchLayer") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

# Leer ALL histórico (full recompute)
raw_events = spark.read \
    .format("parquet") \
    .load("s3://datalake/raw/events/")

# Agregaciones pesadas
user_stats_batch = raw_events \
    .groupBy("user_id", "date") \
    .agg(
        count("*").alias("total_events"),
        sum("revenue").alias("total_revenue"),
        countDistinct("session_id").alias("sessions")
    )

# Escribir a serving layer
user_stats_batch.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("date") \
    .save("s3://datalake/serving/user_stats_batch")

# Schedule: Daily @2AM (Airflow DAG)
# Duration: 2-4 horas para procesar 1 año de datos
# Cost: $$$ (large cluster, pero solo 1x/día)
```

**2. Speed Layer (Low Latency Incremental):**

```python
# Características:
# - Solo datos recientes (últimas horas)
# - Baja latencia (<1 min)
# - Estado mutable (agregaciones incrementales)
# - Complejidad de exactly-once

# Ejemplo: Streaming con Spark Structured Streaming
from pyspark.sql.streaming import StreamingQuery

# Leer solo NUEVOS eventos
events_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "events") \
    .option("startingOffsets", "latest") \
    .load()

# Misma lógica que batch (pero incremental)
user_stats_speed = events_stream \
    .withWatermark("event_timestamp", "10 minutes") \
    .groupBy("user_id", window("event_timestamp", "1 hour")) \
    .agg(
        count("*").alias("total_events"),
        sum("revenue").alias("total_revenue"),
        countDistinct("session_id").alias("sessions")
    )

# Escribir a serving layer (diferentes tablas)
query = user_stats_speed.writeStream \
    .format("delta") \
    .outputMode("update") \
    .option("checkpointLocation", "/checkpoints/speed") \
    .start("s3://datalake/serving/user_stats_speed")

# Runs: Continuously
# Latency: 30s - 2 min
# Cost: $$ (small cluster, pero 24/7)
```

**3. Serving Layer (Query Merge):**

```python
# Merge batch + speed en query time

from datetime import datetime, timedelta

def get_user_stats(user_id: str, as_of: datetime = None):
    """
    Query que merge batch + speed layers
    """
    if as_of is None:
        as_of = datetime.now()
    
    # Batch boundary (típicamente hace 1-2 horas)
    batch_cutoff = as_of - timedelta(hours=2)
    
    # Query batch layer (histórico hasta cutoff)
    batch_stats = spark.sql(f"""
        SELECT 
            user_id,
            SUM(total_events) as events,
            SUM(total_revenue) as revenue
        FROM user_stats_batch
        WHERE user_id = '{user_id}'
          AND date < '{batch_cutoff.date()}'
        GROUP BY user_id
    """).collect()
    
    # Query speed layer (reciente desde cutoff)
    speed_stats = spark.sql(f"""
        SELECT 
            user_id,
            SUM(total_events) as events,
            SUM(total_revenue) as revenue
        FROM user_stats_speed
        WHERE user_id = '{user_id}'
          AND window.start >= '{batch_cutoff}'
        GROUP BY user_id
    """).collect()
    
    # Merge results
    total_events = batch_stats[0]['events'] + speed_stats[0]['events']
    total_revenue = batch_stats[0]['revenue'] + speed_stats[0]['revenue']
    
    return {
        'user_id': user_id,
        'total_events': total_events,
        'total_revenue': total_revenue,
        'as_of': as_of,
        'batch_cutoff': batch_cutoff
    }
```

**Ventajas de Lambda:**

```python
ventajas = {
    "1. Robustez": """
        Batch layer es inmutable → fácil debugging
        Speed layer falla → aún tienes batch como fallback
        Disaster recovery: recompute desde raw data
    """,
    
    "2. Precisión Garantizada": """
        Batch garantiza exactitud (full recompute)
        Speed solo para latencia, batch corrige errores
    """,
    
    "3. Separation of Concerns": """
        Batch: optimiza throughput (large partitions, columnar)
        Speed: optimiza latencia (small batches, in-memory)
        Cada uno usa tecnología ideal para su caso
    """,
    
    "4. Auditoría": """
        Raw data inmutable → compliance (GDPR, SOX)
        Puedes recompute histórico para auditorías
    """
}
```

**Desventajas (¿Por qué Lambda está en declive?):**

```python
desventajas = {
    "1. Código Duplicado": """
        MISMA lógica implementada 2 veces:
        - batch_aggregations.py (Spark Batch)
        - speed_aggregations.py (Spark Streaming)
        
        Cambio de lógica → actualizar AMBOS
        Testing → doble esfuerzo
        Bugs → pueden divergir silenciosamente
    """,
    
    "2. Complejidad Operacional": """
        Mantener 2 pipelines:
        - Batch: Airflow DAG, cluster management, retry logic
        - Speed: Streaming query monitoring, checkpoint management
        
        2x infraestructura, 2x alertas, 2x oncall
    """,
    
    "3. Eventual Consistency": """
        Periodo de gracia donde batch/speed no alineados
        
        Ejemplo:
        10:00 AM: Evento llega
        10:01 AM: Speed layer procesa → visible en dashboard
        02:00 AM: Batch recompute → corrige pequeños errores
        
        Usuario puede ver números ligeramente diferentes
    """,
    
    "4. Costos": """
        Batch cluster: Large (100 nodes) @ 2-4h/día
        Speed cluster: Medium (20 nodes) @ 24/7
        
        Total: ~$50K/mes para org mediana
        
        vs Lakehouse unificado: ~$30K/mes
    """
}
```

**Casos de Uso Reales (Cuándo Lambda Tiene Sentido):**

```python
# 1. LEGACY SYSTEMS con batch existente
"""
Empresa: Retail con 20 años de Hadoop
Situación: 500 Hive tables, cientos de Spark jobs
Necesidad: Agregar real-time sin reescribir todo

Solución: Lambda
- Mantener batch layer existente (no tocas legacy)
- Agregar speed layer con Flink para casos críticos
- Migración gradual dominio por dominio
"""

# 2. ALTA PRECISIÓN requerida
"""
Empresa: Financial trading
Situación: Regulaciones requieren recompute exacto
Necesidad: Auditorías pueden pedir recalcular 5 años atrás

Solución: Lambda
- Batch layer garantiza reproducibilidad exacta
- Speed layer para trading real-time
- Eventual consistency aceptable (batch corrige)
"""

# 3. MUY DIFERENTES SLAs batch vs stream
"""
Empresa: IoT con billones de sensores
Situación: 
  - Alertas críticas: <100ms (anomalías)
  - Análisis histórico: días OK (tendencias)

Solución: Lambda
- Speed layer: Flink para alertas ultra-rápidas
- Batch layer: Hadoop/Hive para análisis pesados
- Tecnologías muy diferentes, difícil unificar
"""
```

**Migración de Lambda → Modern Stack:**

```python
# Estrategia: Unified Batch+Stream con Delta Lake

# ANTES (Lambda):
# 1. Batch: Spark job diario (4h, 100 nodes)
# 2. Speed: Flink continuous (24/7, 20 nodes)
# 3. Serving: Druid (merge queries)

# DESPUÉS (Delta/Lakehouse):
# 1. Unified: Spark Structured Streaming (24/7, 30 nodes)
# 2. Micro-batches cada 5 min
# 3. Delta Lake: ACID transactions, no merge needed

# Migration timeline:
"""
Month 1-2: Setup Delta Lake infrastructure
Month 3-4: Migrate batch pipelines to Delta
Month 5-6: Migrate speed pipelines to Structured Streaming
Month 7: Dual-run (Lambda + Delta) for validation
Month 8: Cutover to Delta, decommission Lambda
Month 9: Cleanup, optimize

Total: 9 months
Cost: $200K engineering + $50K infra
Savings: $20K/mes ongoing (ROI: 2.5 years)
"""
```

**Ejemplo Real: LinkedIn (Inventor de Lambda):**

```python
# LinkedIn inventó Lambda en 2011
# Usaron Lambda 2011-2018 (7 años)

arquitectura_linkedin = {
    "Batch Layer": {
        "Tecnología": "Hadoop MapReduce → Spark",
        "Datos": "Kafka topics replicados a HDFS",
        "Frecuencia": "Daily @midnight",
        "Output": "Hive tables (member profiles, connections, jobs)"
    },
    
    "Speed Layer": {
        "Tecnología": "Storm → Samza → Kafka Streams",
        "Datos": "Kafka topics directamente",
        "Latencia": "<1 second",
        "Output": "Espresso (NoSQL) para low-latency reads"
    },
    
    "Serving Layer": {
        "Tecnología": "Espresso + Voldemort (key-value stores)",
        "Pattern": "API merge batch + speed en read time"
    },
    
    "2018 Migration": """
        LinkedIn migró a Unified Streaming (Samza + Kafka)
        Razón: Complejidad de mantener doble lógica
        Resultado: -30% código, -40% operaciones
    """
}
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [1]:
# 🔄 Simulación Práctica: Arquitectura Lambda
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

print("=" * 80)
print("🏗️ ARQUITECTURA LAMBDA - SIMULACIÓN COMPLETA")
print("=" * 80)

# ============= GENERACIÓN DE DATOS =============
np.random.seed(42)
now = datetime.now()

# Generar 10,000 eventos en los últimos 7 días
events = []
for i in range(10000):
    timestamp = now - timedelta(days=np.random.randint(0, 7), 
                                 hours=np.random.randint(0, 24),
                                 minutes=np.random.randint(0, 60))
    events.append({
        'event_id': f'E{i:06d}',
        'timestamp': timestamp,
        'user_id': f'U{np.random.randint(1, 1000):04d}',
        'event_type': np.random.choice(['view', 'click', 'purchase', 'cart_add']),
        'amount': np.random.uniform(10, 500) if np.random.random() > 0.7 else 0
    })

df_events = pd.DataFrame(events)
print(f"\n📊 Dataset generado: {len(df_events):,} eventos en 7 días")
print(f"Tipos: {df_events['event_type'].value_counts().to_dict()}")

# ============= BATCH LAYER =============
print("\n" + "=" * 80)
print("1️⃣ BATCH LAYER - Procesamiento Completo (simulado overnight)")
print("=" * 80)

start_time = time.time()

# Simular procesamiento batch: agregación por día y usuario
df_batch = df_events.copy()
df_batch['date'] = df_batch['timestamp'].dt.date

batch_results = df_batch.groupby(['date', 'user_id']).agg({
    'event_id': 'count',
    'amount': 'sum'
}).rename(columns={'event_id': 'total_events', 'amount': 'total_revenue'}).reset_index()

batch_time = time.time() - start_time

print(f"✅ Batch completo procesado en {batch_time:.3f}s")
print(f"📈 Agregaciones generadas: {len(batch_results):,} registros (user-day level)")
print(f"💰 Revenue total: ${batch_results['total_revenue'].sum():,.2f}")
print(f"\nÚltima fecha batch: {batch_results['date'].max()}")

# ============= SPEED LAYER =============
print("\n" + "=" * 80)
print("2️⃣ SPEED LAYER - Streaming en tiempo real")
print("=" * 80)

# Simular eventos recientes (última hora)
cutoff_time = now - timedelta(hours=1)
df_speed = df_events[df_events['timestamp'] > cutoff_time].copy()

start_time = time.time()
speed_results = df_speed.groupby('user_id').agg({
    'event_id': 'count',
    'amount': 'sum'
}).rename(columns={'event_id': 'events_last_hour', 'amount': 'revenue_last_hour'}).reset_index()

speed_time = time.time() - start_time

print(f"⚡ Speed layer procesado en {speed_time:.3f}s")
print(f"📊 Eventos recientes: {len(df_speed):,} en última hora")
print(f"👥 Usuarios activos: {len(speed_results):,}")
print(f"💰 Revenue última hora: ${speed_results['revenue_last_hour'].sum():,.2f}")

# ============= SERVING LAYER =============
print("\n" + "=" * 80)
print("3️⃣ SERVING LAYER - Merge de Batch + Speed")
print("=" * 80)

# Simular consulta: obtener métricas completas para usuarios
sample_users = df_events['user_id'].sample(5).unique()

for user in sample_users:
    # Batch data (histórico hasta hace 1 hora)
    user_batch = batch_results[batch_results['user_id'] == user]
    batch_events = user_batch['total_events'].sum() if len(user_batch) > 0 else 0
    batch_revenue = user_batch['total_revenue'].sum() if len(user_batch) > 0 else 0
    
    # Speed data (última hora)
    user_speed = speed_results[speed_results['user_id'] == user]
    speed_events = user_speed['events_last_hour'].iloc[0] if len(user_speed) > 0 else 0
    speed_revenue = user_speed['revenue_last_hour'].iloc[0] if len(user_speed) > 0 else 0
    
    # MERGE
    total_events = batch_events + speed_events
    total_revenue = batch_revenue + speed_revenue
    
    print(f"\n👤 {user}:")
    print(f"   Batch (histórico): {batch_events} eventos, ${batch_revenue:.2f}")
    print(f"   Speed (real-time): {speed_events} eventos, ${speed_revenue:.2f}")
    print(f"   ➡️ TOTAL: {total_events} eventos, ${total_revenue:.2f}")

# ============= TRADE-OFFS =============
print("\n" + "=" * 80)
print("📊 ANÁLISIS DE TRADE-OFFS")
print("=" * 80)

trade_offs = pd.DataFrame([
    {'Aspecto': 'Latencia', 'Batch Layer': '6-24 horas', 'Speed Layer': '<1 segundo', 'Impacto': '⚠️ Queries complejas (merge)'},
    {'Aspecto': 'Precisión', 'Batch Layer': '100% preciso', 'Speed Layer': 'Aproximado', 'Impacto': '✅ Eventual consistency'},
    {'Aspecto': 'Complejidad', 'Batch Layer': 'Simple (MapReduce)', 'Speed Layer': 'Complejo (estado)', 'Impacto': '⚠️ Doble lógica'},
    {'Aspecto': 'Costo operacional', 'Batch Layer': 'Bajo (batch)', 'Speed Layer': 'Alto (24/7)', 'Impacto': '💰 2x infraestructura'},
])

print(trade_offs.to_string(index=False))

print(f"\n" + "=" * 80)
print("✅ Simulación Lambda completada")
print(f"   • Batch procesó {len(df_events):,} eventos históricos")
print(f"   • Speed procesó {len(df_speed):,} eventos en tiempo real")
print(f"   • Serving layer unificó resultados con éxito")
print("=" * 80)

🏗️ ARQUITECTURA LAMBDA - SIMULACIÓN COMPLETA

📊 Dataset generado: 10,000 eventos en 7 días
Tipos: {np.str_('view'): 2521, np.str_('purchase'): 2512, np.str_('click'): 2502, np.str_('cart_add'): 2465}

1️⃣ BATCH LAYER - Procesamiento Completo (simulado overnight)
✅ Batch completo procesado en 0.009s
📈 Agregaciones generadas: 5,577 registros (user-day level)
💰 Revenue total: $758,138.30

Última fecha batch: 2025-12-09

2️⃣ SPEED LAYER - Streaming en tiempo real
⚡ Speed layer procesado en 0.003s
📊 Eventos recientes: 55 en última hora
👥 Usuarios activos: 54
💰 Revenue última hora: $3,225.21

3️⃣ SERVING LAYER - Merge de Batch + Speed

👤 U0385:
   Batch (histórico): 6 eventos, $334.87
   Speed (real-time): 0 eventos, $0.00
   ➡️ TOTAL: 6 eventos, $334.87

👤 U0270:
   Batch (histórico): 14 eventos, $1383.49
   Speed (real-time): 0 eventos, $0.00
   ➡️ TOTAL: 14 eventos, $1383.49

👤 U0456:
   Batch (histórico): 10 eventos, $921.91
   Speed (real-time): 0 eventos, $0.00
   ➡️ TOTAL: 10 eventos,

### 🔄 **Kappa Architecture: Stream-Only Simplification**

**Origen (Jay Kreps, LinkedIn/Confluent, 2014):**

```
Crítica a Lambda:
"¿Por qué mantener 2 sistemas si streaming puede hacer todo?"

Propuesta Kappa:
- Eliminar batch layer completamente
- Todo es streaming (batch = replay del log)
- Inmutable log como single source of truth
```

**Arquitectura:**

```python
┌─────────────────────────────────────────┐
│         DATA SOURCES                     │
│  • Applications                          │
│  • Databases (CDC)                       │
│  • IoT Devices                           │
└─────────────┬───────────────────────────┘
              │
              ▼
┌─────────────────────────────────────────┐
│    KAFKA (Immutable Log)                │
│                                          │
│  • Infinite retention (or very long)    │
│  • Partitioned & Replicated             │
│  • Serves as "Master Dataset"           │
│                                          │
│  Topics:                                 │
│  ├─ events (retention: 1 year)          │
│  ├─ transactions (retention: 5 years)   │
│  └─ user_actions (retention: 90 days)   │
└─────────────┬───────────────────────────┘
              │
              ▼
┌─────────────────────────────────────────┐
│   STREAM PROCESSING                     │
│                                          │
│  Version 1: Flink Job                   │
│  ├─ Aggregations                        │
│  ├─ Joins                               │
│  └─ Output → Cassandra                  │
│                                          │
│  Version 2: New logic needed            │
│  ├─ Deploy new Flink job                │
│  ├─ Replay from offset 0                │
│  └─ Output → Cassandra v2               │
│                                          │
│  🔄 Reprocessing = Replay log           │
└─────────────┬───────────────────────────┘
              │
              ▼
┌─────────────────────────────────────────┐
│       SERVING LAYER                     │
│  • Cassandra                            │
│  • ElasticSearch                        │
│  • Redis                                │
│  • PostgreSQL                           │
└─────────────────────────────────────────┘
```

**Principios Clave:**

```python
principios_kappa = {
    "1. Everything is a Stream": """
        No distinción entre batch y stream
        
        Batch tradicional:
        SELECT SUM(revenue) FROM sales WHERE date = '2025-10-30'
        
        Kappa:
        Consume Kafka topic 'sales', aggregate, done
        (No diferencia conceptual con real-time)
    """,
    
    "2. Immutable Log as Source of Truth": """
        Kafka = Database of record
        
        Ventajas:
        - Time travel: replay desde cualquier offset
        - Debugging: reproduce bug con datos exactos
        - Migration: deploy nueva versión, replay, compare
        
        Ejemplo:
        offset 0 → offset 1M (1 año de datos)
        Reprocessing: ~4 horas con 50 partitions
    """,
    
    "3. Schema Evolution in Log": """
        Log contiene todos los schemas históricos
        
        V1: {"user_id": 123, "action": "click"}
        V2: {"user_id": 123, "action": "click", "device": "mobile"}
        V3: {"user_id": 123, "event_type": "click", "metadata": {...}}
        
        Consumer handle all versions gracefully
    """,
    
    "4. Reprocessing for Code Changes": """
        Cambio de lógica → no reescribir batch
        
        Workflow:
        1. Deploy new stream processor (version 2)
        2. Replay desde offset 0 (parallel con v1)
        3. Validate output v2 matches expected
        4. Cutover traffic to v2
        5. Decommission v1
        
        Duration: horas/días (no semanas de reescribir batch)
    """
}
```

**Implementación Real con Kafka + Flink:**

```python
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment
from pyflink.table.descriptors import Kafka, Schema, Json

env = StreamExecutionEnvironment.get_execution_environment()
t_env = StreamTableEnvironment.create(env)

# Configuración Kafka source con retención larga
t_env.execute_sql("""
    CREATE TABLE events (
        user_id STRING,
        event_type STRING,
        product_id STRING,
        revenue DOUBLE,
        event_timestamp TIMESTAMP(3),
        WATERMARK FOR event_timestamp AS event_timestamp - INTERVAL '10' SECOND
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'events',
        'properties.bootstrap.servers' = 'kafka:9092',
        'properties.group.id' = 'user-stats-processor-v2',
        'scan.startup.mode' = 'earliest-offset',  -- Replay desde inicio
        'format' = 'json',
        'json.timestamp-format.standard' = 'ISO-8601'
    )
""")

# Stream processing (unificado batch+stream)
user_stats = t_env.sql_query("""
    SELECT 
        user_id,
        TUMBLE_START(event_timestamp, INTERVAL '1' DAY) as day,
        COUNT(*) as total_events,
        SUM(CASE WHEN event_type = 'purchase' THEN revenue ELSE 0 END) as total_revenue,
        COUNT(DISTINCT product_id) as unique_products
    FROM events
    GROUP BY 
        user_id,
        TUMBLE(event_timestamp, INTERVAL '1' DAY)
""")

# Output a Cassandra
t_env.execute_sql("""
    CREATE TABLE user_stats_cassandra (
        user_id STRING,
        day TIMESTAMP(3),
        total_events BIGINT,
        total_revenue DOUBLE,
        unique_products BIGINT,
        PRIMARY KEY (user_id, day) NOT ENFORCED
    ) WITH (
        'connector' = 'cassandra',
        'host' = 'cassandra',
        'table-name' = 'user_stats'
    )
""")

user_stats.execute_insert('user_stats_cassandra').wait()

# Reprocessing:
# 1. Stop old job
# 2. Deploy this code (nueva lógica)
# 3. Kafka consumer group lee desde offset 0
# 4. Reprocesa 1 año de datos en ~4 horas
# 5. Nueva tabla user_stats_cassandra tiene datos corregidos
```

**Comparación Kafka Retention:**

```python
# Configuración típica para Kappa
kafka_retention = {
    "Short Retention (Anti-Kappa)": {
        "Config": "retention.ms = 604800000  # 7 días",
        "Disk": "1 TB / partition",
        "Cost": "$500/mes",
        "Reprocessing": "❌ Solo últimos 7 días disponibles",
        "Use Case": "Lambda architecture (Kafka solo buffer)"
    },
    
    "Long Retention (Kappa)": {
        "Config": "retention.ms = 31536000000  # 1 año",
        "Disk": "52 TB / partition (con compaction)",
        "Cost": "$2,600/mes (S3 tiered storage)",
        "Reprocessing": "✅ 1 año completo disponible",
        "Use Case": "Kappa architecture (Kafka es source of truth)"
    },
    
    "Infinite Retention (Extreme Kappa)": {
        "Config": "retention.ms = -1  # Forever",
        "Disk": "Ilimitado (requiere tiered storage)",
        "Cost": "$5K/mes (S3 archival)",
        "Reprocessing": "✅ Histórico completo",
        "Use Case": "Event sourcing, auditabilidad estricta"
    }
}

# Kafka Tiered Storage (KIP-405, Kafka 3.0+)
# Archiva logs antiguos a S3, mantiene recientes en SSD
"""
kafka-storage.properties:
  remote.log.storage.enable=true
  remote.log.storage.manager.class=org.apache.kafka.server.log.remote.storage.RemoteLogStorageManager
  
Estructura:
  Local SSD: Últimos 7 días (hot data)
  S3 Bucket: >7 días (cold data, comprimido)
  
Cost reduction: 70% vs all-SSD
"""
```

**Ventajas de Kappa:**

```python
ventajas_kappa = {
    "1. Simplicidad Operacional": """
        Un solo pipeline → un solo sistema para monitorear
        Un solo lenguaje/framework (Flink o Spark)
        Un solo cluster para operar
        
        On-call rotation:
        Lambda: 2 sistemas × 2 equipos = 4 rotaciones/semana
        Kappa: 1 sistema × 1 equipo = 2 rotaciones/semana
    """,
    
    "2. No Código Duplicado": """
        Misma lógica para batch y stream
        
        Lambda: 2000 líneas batch.py + 2000 líneas stream.py = 4000
        Kappa: 2000 líneas unified.py = 2000 (50% menos código)
        
        Bug fix: 1 cambio vs 2 cambios
        Testing: 1 suite vs 2 suites
    """,
    
    "3. Reprocessing Rápido": """
        Cambio de lógica → replay log en horas
        
        Lambda:
        - Reescribir batch job (días de dev)
        - Ejecutar en cluster grande (4h)
        - Validar, debug, iterar
        Total: 1-2 semanas
        
        Kappa:
        - Modificar stream job (horas de dev)
        - Replay desde offset 0 (4h)
        - Compare old vs new output
        Total: 1 día
    """,
    
    "4. Exactly-Once Más Simple": """
        Kafka transactions + idempotent producers
        
        Flink + Kafka:
        - Checkpointing en Kafka offsets
        - Transactional writes a sinks
        - No necesidad de merge batch+speed
        
        Lambda:
        - Batch tiene at-least-once (reintentos)
        - Speed tiene exactly-once (complejo)
        - Merge puede duplicar sin deduplication
    """
}
```

**Desventajas de Kappa:**

```python
desventajas_kappa = {
    "1. Costo de Almacenamiento": """
        Kafka con 1 año de retención = caro
        
        Ejemplo: 10K events/s × 10 KB/event × 1 año
        = 3.15 PB/año crudo
        Con compaction + tiered storage: 500 TB
        Cost: $5K/mes S3 + $10K/mes Kafka brokers
        
        vs Hadoop HDFS: $2K/mes (pero sin streaming)
    """,
    
    "2. Reprocessing Largo": """
        Replay 1 año de datos = horas/días
        
        1 TB data @ 100 MB/s = 2.8 horas ideal
        Real (with processing): 6-12 horas
        
        Durante reprocessing:
        - Nuevo código corre en paralelo (doble costo)
        - Output dual (old version + new version)
        - Validación manual necesaria
        
        Lambda:
        - Batch recompute overnight (no impact users)
        - Speed layer sigue sirviendo tráfico
    """,
    
    "3. State Management Complejo": """
        Streaming state para 1 año de datos = grande
        
        Ejemplo: User profiles con aggregations
        10M users × 1 KB state = 10 GB state
        
        Flink RocksDB:
        - State en disco (slower)
        - Checkpointing largo (minutes)
        - Recovery largo si crash (restore state)
        
        Lambda batch:
        - No state (stateless join con full tables)
        - Simpler, pero más lento
    """,
    
    "4. No Optimización por Caso": """
        Streaming debe servir TODOS los casos
        
        Lambda:
        - Batch: Columnar Parquet, predicate pushdown, Z-order
        - Speed: Row-based, in-memory, indexes
        
        Kappa:
        - Un formato compromiso (no óptimo para ninguno)
        - Queries complejos (joins 5 tables) lentos en stream
        - Queries simples (count) overkill con stream
    """
}
```

**Casos de Uso Ideales para Kappa:**

```python
# 1. REAL-TIME FIRST con poco histórico
"""
Empresa: Fintech fraud detection
Datos: 90 días de transacciones
Procesamiento: <100ms latency requerido
Reprocessing: Semanal (ajustar modelos ML)

Kappa perfecto:
- 90 días en Kafka (100 TB, manejable)
- Flink streaming para detección real-time
- Replay semanal para ajustar modelos (4h)
"""

# 2. EVENT SOURCING puro
"""
Empresa: Booking platform
Patrón: Event sourcing (todo es evento inmutable)
Datos: bookings, cancellations, modifications
Queries: Reconstruir estado actual desde eventos

Kappa natural:
- Kafka como event store (immutable log)
- Flink materializa vistas (current bookings)
- Replay para nuevas vistas (add "bookings by hotel")
"""

# 3. HIGH CHANGE FREQUENCY en lógica
"""
Empresa: Ad-tech con modelos A/B testing constante
Situación: Cambio de lógica 2-3x/semana
Necesidad: Deploy rápido, validar, iterar

Kappa ventaja:
- Modify stream job (1h dev)
- Deploy v2 parallel v1 (30 min)
- Replay último día (30 min)
- Compare outputs, cutover (1h)
Total: 3 horas por cambio vs 3 días Lambda
"""

# 4. SMALL-MEDIUM data scale
"""
Empresa: SaaS startup
Datos: <100 TB total
Users: <1M
Traffic: <1K events/s

Kappa ventaja:
- Un cluster Kafka+Flink (simple)
- Retención 1 año fácil (10 TB en Kafka)
- Equipo pequeño puede mantener
"""
```

**Anti-Patterns (Cuándo NO usar Kappa):**

```python
# ❌ 1. HUGE data scale con queries complejos
"""
Empresa: Retail con 10+ años de histórico
Datos: 100 PB en HDFS
Queries: SQL complejo (20+ joins, window functions)

Problema Kappa:
- 100 PB en Kafka = $500K/mes (vs $50K HDFS)
- Streaming joins lentos vs batch columnar
- Replay 10 años = semanas (inviable)

Better: Lambda o Delta Lakehouse
"""

# ❌ 2. BATCH-FIRST workloads
"""
Empresa: Data warehouse con reportes nocturnos
Queries: ETL batch 90% workload, streaming 10%
Latency: Horas OK para reportes

Problema Kappa:
- Overhead streaming para batch workloads
- Kafka infraestructura innecesaria
- Spark batch más eficiente que Spark Streaming para batch

Better: Tradicional batch (Airflow + Spark)
"""

# ❌ 3. ESTRICTA compliance con reprocessing prohibido
"""
Empresa: Banco con regulaciones estrictas
Requerimiento: Auditorías requieren datos exactos "as-of"
Prohibición: No se puede "rehacer" cálculos pasados

Problema Kappa:
- Reprocessing es core feature (pero regulador dice no)
- Inmutabilidad requiere batch layer aparte

Better: Lambda con batch layer auditado/certificado
"""
```

**Ejemplo Real: Uber (Kappa en Producción):**

```python
uber_kappa = {
    "Caso": "Surge pricing (precios dinámicos)",
    
    "Arquitectura": {
        "Eventos": "ride_requests, driver_locations, ride_completions",
        "Kafka": "500K events/s, retención 7 días (suficiente)",
        "Flink": "Windowed aggregations (supply/demand por área)",
        "Output": "Redis (precios actuales), Cassandra (histórico)",
        "Latency": "<200ms end-to-end"
    },
    
    "Kappa Benefits": """
        1. Cambio frecuente de algoritmo pricing (weekly)
        2. Replay 7 días para validar nuevo algoritmo (2h)
        3. A/B testing: correr 2 versiones paralelo, compare
        4. No batch layer needed (solo últimos días relevantes)
    """,
    
    "Evolution": """
        2015: Lambda (Spark batch + Storm streaming)
        2016: Migrated to Kappa (Samza streaming only)
        2018: Moved to Flink (better stateful processing)
        2020: Hybrid (Kappa para real-time, Hudi para analytics)
        
        Razón hybrid: Analytics largo plazo necesita batch optimizations
    """
}
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [2]:
# 🔄 Simulación Práctica: Arquitectura Kappa
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

print("=" * 80)
print("♾️ ARQUITECTURA KAPPA - STREAMING UNIFICADO")
print("=" * 80)

# ============= KAFKA LOG INMUTABLE =============
np.random.seed(42)
now = datetime.now()

# Simular Kafka log con 30 días de retención
print("\n📚 Kafka Topic: 'events' (retention: 30 días)")
events_log = []
for i in range(15000):
    timestamp = now - timedelta(days=np.random.randint(0, 30),
                                 hours=np.random.randint(0, 24),
                                 minutes=np.random.randint(0, 60))
    events_log.append({
        'offset': i,
        'timestamp': timestamp,
        'user_id': f'U{np.random.randint(1, 800):04d}',
        'event_type': np.random.choice(['view', 'click', 'purchase', 'cart_add']),
        'amount': np.random.uniform(10, 500) if np.random.random() > 0.7 else 0
    })

df_kafka = pd.DataFrame(events_log).sort_values('timestamp').reset_index(drop=True)
df_kafka['offset'] = range(len(df_kafka))

print(f"✅ Log almacenado: {len(df_kafka):,} eventos")
print(f"📅 Rango temporal: {df_kafka['timestamp'].min().date()} → {df_kafka['timestamp'].max().date()}")
print(f"💾 Storage aprox: {len(df_kafka) * 0.5 / 1024:.2f} MB (50% compresión)")

# ============= STREAMING JOB V1 (Producción actual) =============
print("\n" + "=" * 80)
print("1️⃣ STREAMING JOB V1 - Procesamiento en tiempo real")
print("=" * 80)

start_time = time.time()

# Simular consumo desde último offset (streaming continuo)
current_offset = len(df_kafka) - 100  # Últimos 100 eventos
df_streaming = df_kafka[df_kafka['offset'] >= current_offset].copy()

# Agregaciones en ventanas de tiempo (micro-batches)
df_streaming['window'] = df_streaming['timestamp'].dt.floor('5min')
stream_results = df_streaming.groupby(['window', 'user_id']).agg({
    'offset': 'count',
    'amount': 'sum'
}).rename(columns={'offset': 'events', 'amount': 'revenue'}).reset_index()

stream_time = time.time() - start_time

print(f"⚡ Streaming job procesando desde offset {current_offset:,}")
print(f"📊 Eventos procesados: {len(df_streaming):,}")
print(f"⏱️ Latencia promedio: {stream_time * 1000 / len(df_streaming):.2f}ms por evento")
print(f"💰 Revenue acumulado: ${stream_results['revenue'].sum():,.2f}")
print(f"👥 Usuarios activos: {df_streaming['user_id'].nunique()}")

# ============= REPROCESSING (Nueva lógica) =============
print("\n" + "=" * 80)
print("2️⃣ REPROCESSING - Deploy nueva versión (replay completo)")
print("=" * 80)

print("🔄 Escenario: Nueva lógica requiere recalcular TODOS los resultados")
print("   • Cambio: Agregar detección de anomalías en compras")
print("   • Solución Kappa: Replay del log completo desde offset 0\n")

start_time = time.time()

# Replay completo del log
df_replay = df_kafka.copy()

# Nueva lógica: detectar compras anómalas (>3x el promedio del usuario)
user_avg = df_replay[df_replay['amount'] > 0].groupby('user_id')['amount'].mean()
df_replay['user_avg_purchase'] = df_replay['user_id'].map(user_avg)
df_replay['is_anomaly'] = (df_replay['amount'] > 0) & (df_replay['amount'] > df_replay['user_avg_purchase'] * 3)

anomalies = df_replay[df_replay['is_anomaly'] == True]

# Reagregación con nueva lógica
replay_results = df_replay.groupby('user_id').agg({
    'offset': 'count',
    'amount': 'sum',
    'is_anomaly': 'sum'
}).rename(columns={'offset': 'total_events', 'amount': 'total_revenue', 'is_anomaly': 'anomaly_count'}).reset_index()

replay_time = time.time() - start_time

print(f"✅ Replay completado en {replay_time:.2f}s")
print(f"📈 Eventos reprocesados: {len(df_replay):,} (100% del log)")
print(f"🚨 Anomalías detectadas: {len(anomalies):,} ({len(anomalies)/len(df_replay)*100:.2f}%)")
print(f"💰 Total revenue: ${replay_results['total_revenue'].sum():,.2f}")
print(f"👥 Usuarios con anomalías: {replay_results[replay_results['anomaly_count'] > 0].shape[0]}")

# Ejemplo de usuarios con anomalías
print("\n📋 Top 5 usuarios con más anomalías:")
top_anomalies = replay_results.nlargest(5, 'anomaly_count')[['user_id', 'total_events', 'anomaly_count', 'total_revenue']]
for _, row in top_anomalies.iterrows():
    print(f"   {row['user_id']}: {int(row['anomaly_count'])} anomalías de {int(row['total_events'])} eventos (${row['total_revenue']:.2f})")

# ============= COMPARACIÓN CON LAMBDA =============
print("\n" + "=" * 80)
print("📊 KAPPA vs LAMBDA")
print("=" * 80)

comparison = pd.DataFrame([
    {
        'Aspecto': 'Código',
        'Lambda': '2 codebases (batch + stream)',
        'Kappa': '1 codebase (stream)',
        'Ganador': '✅ Kappa'
    },
    {
        'Aspecto': 'Reprocessing',
        'Lambda': 'Batch separado (MapReduce)',
        'Kappa': 'Replay del log (mismo pipeline)',
        'Ganador': '✅ Kappa'
    },
    {
        'Aspecto': 'Latencia',
        'Lambda': 'Speed: <1s, Batch: horas',
        'Kappa': 'Siempre <1s',
        'Ganador': '✅ Kappa'
    },
    {
        'Aspecto': 'Costo storage',
        'Lambda': 'Batch: S3, Speed: memoria',
        'Kappa': 'Kafka log (costoso)',
        'Ganador': '⚠️ Lambda'
    },
    {
        'Aspecto': 'Estado complejo',
        'Lambda': 'Batch: stateless',
        'Kappa': 'Stream: stateful (RocksDB)',
        'Ganador': '⚠️ Lambda'
    }
])

print(comparison.to_string(index=False))

# ============= TRADE-OFFS ESPECÍFICOS =============
print("\n" + "=" * 80)
print("⚖️ TRADE-OFFS DE KAPPA")
print("=" * 80)

trade_offs = {
    'Ventajas': [
        '✅ Simplicidad operacional (1 sistema)',
        '✅ Consistencia de lógica (sin duplicación)',
        '✅ Latencia baja para todo',
        '✅ Reprocessing con mismo código'
    ],
    'Desventajas': [
        '⚠️ Kafka storage costoso (30+ días de eventos)',
        '⚠️ Estado complejo en streaming (checkpoints grandes)',
        '⚠️ Replay lento para históricos masivos (años)',
        '⚠️ No optimizado para queries batch complejos'
    ]
}

for category, items in trade_offs.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  {item}")

print(f"\n" + "=" * 80)
print("✅ Simulación Kappa completada")
print(f"   • Log de Kafka: {len(df_kafka):,} eventos (30 días)")
print(f"   • Streaming V1 procesó {len(df_streaming):,} eventos en tiempo real")
print(f"   • Reprocessing V2 replayó {len(df_replay):,} eventos en {replay_time:.2f}s")
print(f"   • Anomalías detectadas: {len(anomalies):,}")
print("=" * 80)

♾️ ARQUITECTURA KAPPA - STREAMING UNIFICADO

📚 Kafka Topic: 'events' (retention: 30 días)
✅ Log almacenado: 15,000 eventos
📅 Rango temporal: 2025-11-09 → 2025-12-09
💾 Storage aprox: 7.32 MB (50% compresión)

1️⃣ STREAMING JOB V1 - Procesamiento en tiempo real
⚡ Streaming job procesando desde offset 14,900
📊 Eventos procesados: 100
⏱️ Latencia promedio: 0.13ms por evento
💰 Revenue acumulado: $6,046.20
👥 Usuarios activos: 97

2️⃣ REPROCESSING - Deploy nueva versión (replay completo)
🔄 Escenario: Nueva lógica requiere recalcular TODOS los resultados
   • Cambio: Agregar detección de anomalías en compras
   • Solución Kappa: Replay del log completo desde offset 0

✅ Replay completado en 0.01s
📈 Eventos reprocesados: 15,000 (100% del log)
🚨 Anomalías detectadas: 3 (0.02%)
💰 Total revenue: $1,142,590.63
👥 Usuarios con anomalías: 3

📋 Top 5 usuarios con más anomalías:
   U0184: 1 anomalías de 15 eventos ($942.03)
   U0373: 1 anomalías de 22 eventos ($957.25)
   U0590: 1 anomalías de 20 evento

### 🏠 **Delta Architecture (Lakehouse): Unified Batch+Stream**

**Evolución Natural (2020+):**

```
Lambda (2011): Batch + Speed layers separados
   ↓
Kappa (2014): Solo streaming (pero caro, complejo)
   ↓
Delta (2020): Unified sobre transactional storage
```

**Arquitectura Lakehouse:**

```python
┌─────────────────────────────────────────────────────────────┐
│                    DATA SOURCES                              │
│  • Applications → Kafka                                      │
│  • Databases → CDC (Debezium)                                │
│  • Batch files → S3/ADLS                                     │
└────────────────────┬────────────────────────────────────────┘
                     │
                     ▼
          ┌──────────────────┐
          │  INGESTION LAYER │
          │                  │
          │  Spark Streaming │ ← Unified engine!
          │  (micro-batches) │
          └────────┬─────────┘
                   │
                   ▼
┌──────────────────────────────────────────────────────────────┐
│              DELTA LAKE / ICEBERG STORAGE                     │
│                                                               │
│  Bronze (Raw)           Silver (Cleaned)      Gold (Curated) │
│  ┌────────────┐        ┌────────────┐       ┌────────────┐  │
│  │ Append-only│        │ Deduplicated│      │Aggregations│  │
│  │ Parquet +  │   →    │ Validated   │  →   │Star Schema │  │
│  │ Delta Log  │        │ Enriched    │      │Business    │  │
│  │            │        │             │      │Logic       │  │
│  └────────────┘        └────────────┘       └────────────┘  │
│                                                               │
│  Features:                                                    │
│  ✅ ACID Transactions                                        │
│  ✅ Time Travel (rollback, audit)                           │
│  ✅ Schema Evolution                                         │
│  ✅ Upserts/Deletes                                          │
│  ✅ Unified Batch+Stream reads                               │
└────────────────────┬──────────────────────────────────────────┘
                     │
          ┌──────────┴─────────┐
          │                    │
          ▼                    ▼
  ┌───────────────┐    ┌──────────────┐
  │ BATCH QUERIES │    │  STREAMING   │
  │               │    │   QUERIES    │
  │ Spark SQL     │    │ Spark SS     │
  │ Presto/Trino  │    │ Flink        │
  │ Athena        │    │ Real-time    │
  └───────────────┘    └──────────────┘
          │                    │
          └──────────┬─────────┘
                     ▼
            ┌─────────────────┐
            │   CONSUMPTION   │
            │                 │
            │  • BI Tools     │
            │  • ML Training  │
            │  • APIs         │
            │  • Dashboards   │
            └─────────────────┘
```

**Características Clave:**

```python
delta_features = {
    "1. ACID Transactions en Object Storage": """
        Problema tradicional:
        S3/ADLS no tienen transacciones
        → Race conditions, partial writes
        
        Delta Lake solution:
        _delta_log/00000000000000000123.json
        {
          "commitInfo": {"timestamp": "2025-10-30T10:30:00"},
          "add": [
            {"path": "part-00000.parquet", "size": 1024000},
            {"path": "part-00001.parquet", "size": 1024000}
          ],
          "remove": []
        }
        
        Atomic commit: Write log entry = commit completo
        Readers ven versión consistente (snapshot isolation)
    """,
    
    "2. Unified Batch + Stream": """
        SAME TABLE para batch y streaming
        
        Escritura streaming:
        df.writeStream
          .format("delta")
          .outputMode("append")
          .start("/delta/events")
        
        Lectura batch:
        spark.read
          .format("delta")
          .load("/delta/events")
          .where("date = '2025-10-30'")
        
        Lectura streaming:
        spark.readStream
          .format("delta")
          .load("/delta/events")  # Lee cambios incrementales
        
        NO necesitas batch layer separada!
    """,
    
    "3. Time Travel": """
        Acceso a versiones históricas
        
        # Leer versión específica
        df = spark.read
          .format("delta")
          .option("versionAsOf", 42)
          .load("/delta/events")
        
        # Leer timestamp específico
        df = spark.read
          .format("delta")
          .option("timestampAsOf", "2025-10-29 00:00:00")
          .load("/delta/events")
        
        Casos de uso:
        - Rollback: Deployment malo → revert
        - Audit: "¿Qué datos vio el modelo ayer?"
        - Debugging: "Reproduce bug con datos exactos"
        - Compliance: "Muestra datos as-of Q3 2025"
    """,
    
    "4. Schema Evolution": """
        Agregar columnas sin downtime
        
        V1: {user_id, action, timestamp}
        V2: {user_id, action, timestamp, device}  ← Add column
        
        Delta:
        df_v2.write
          .format("delta")
          .mode("append")
          .option("mergeSchema", "true")
          .save("/delta/events")
        
        Lectores antiguos: device = null
        Lectores nuevos: device leído correctamente
        
        NO necesitas backfill!
    """,
    
    "5. Upserts (MERGE)": """
        CDC y SCD Type 2 nativos
        
        from delta.tables import DeltaTable
        
        delta_table = DeltaTable.forPath(spark, "/delta/users")
        
        delta_table.alias("target").merge(
            updates.alias("source"),
            "target.user_id = source.user_id"
        ).whenMatchedUpdate(set={
            "email": "source.email",
            "updated_at": "source.updated_at"
        }).whenNotMatchedInsert(values={
            "user_id": "source.user_id",
            "email": "source.email",
            "created_at": "source.created_at",
            "updated_at": "source.updated_at"
        }).execute()
        
        Streaming UPSERT:
        updates.writeStream
          .foreachBatch(lambda df, _: upsert_function(df))
          .start()
    """
}
```

**Medallion Architecture (Bronze/Silver/Gold):**

```python
# BRONZE: Raw data, append-only
bronze_events = spark.readStream \
    .format("kafka") \
    .option("subscribe", "events") \
    .load() \
    .select(
        col("value").cast("string").alias("raw_json"),
        col("timestamp").alias("ingestion_time")
    )

bronze_events.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/checkpoints/bronze") \
    .option("mergeSchema", "true") \
    .start("/delta/bronze/events")

# SILVER: Cleaned, validated, deduplicated
from pyspark.sql.functions import from_json, col

event_schema = StructType([
    StructField("user_id", StringType()),
    StructField("event_type", StringType()),
    StructField("event_timestamp", TimestampType())
])

silver_events = spark.readStream \
    .format("delta") \
    .load("/delta/bronze/events") \
    .select(
        from_json(col("raw_json"), event_schema).alias("data")
    ) \
    .select("data.*") \
    .filter(col("user_id").isNotNull()) \  # Validación
    .dropDuplicates(["user_id", "event_timestamp"])  # Dedup

silver_events.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/checkpoints/silver") \
    .start("/delta/silver/events")

# GOLD: Business aggregations
gold_user_stats = spark.readStream \
    .format("delta") \
    .load("/delta/silver/events") \
    .groupBy("user_id", window("event_timestamp", "1 day")) \
    .agg(
        count("*").alias("daily_events"),
        countDistinct("event_type").alias("event_types")
    )

gold_user_stats.writeStream \
    .format("delta") \
    .outputMode("update") \
    .option("checkpointLocation", "/checkpoints/gold") \
    .start("/delta/gold/user_daily_stats")

# BI Tool lee desde Gold
df_dashboard = spark.read \
    .format("delta") \
    .load("/delta/gold/user_daily_stats") \
    .where("window.start >= current_date() - interval 7 days")
```

**Ventajas sobre Lambda/Kappa:**

```python
ventajas_delta = {
    "vs Lambda": {
        "Código Unificado": """
            Lambda: 2 pipelines (batch.py + stream.py)
            Delta: 1 pipeline (unified.py)
            
            Reduction: 50% código, 50% tests, 50% bugs
        """,
        
        "Sin Merge Layer": """
            Lambda: Query = merge(batch, speed) en serving
            Delta: Query directo a Delta (single source)
            
            Latency: -100ms (no merge overhead)
            Consistency: Guaranteed (ACID transactions)
        """,
        
        "Operaciones Simples": """
            Lambda: Mantener 2 clusters + serving layer
            Delta: 1 cluster Spark + Delta storage
            
            On-call: -50% alerts, -40% incidents
        """
    },
    
    "vs Kappa": {
        "Costo Storage": """
            Kappa: Kafka 1 año = $15K/mes
            Delta: S3 object storage = $2K/mes
            
            Savings: 85% storage costs
        """,
        
        "Batch Optimized": """
            Kappa: Streaming para todo (no óptimo)
            Delta: Columnar Parquet + statistics
            
            Query performance: 10x faster para analytics
        """,
        
        "Reprocessing Flexible": """
            Kappa: Replay Kafka (limited retention)
            Delta: Time travel infinito
            
            Auditabilidad: 5+ años histórico disponible
        """
    },
    
    "vs Traditional Warehouse": {
        "Cost": """
            Warehouse: $25/TB/month (Snowflake)
            Delta: $0.023/GB/month (S3) = $23/TB
            
            Savings: ~50% at petabyte scale
        """,
        
        "Flexibility": """
            Warehouse: Vendor lock-in, SQL only
            Delta: Open format, Spark/Presto/Flink/Python
            
            ML Integration: Native (same storage)
        """,
        
        "Scalability": """
            Warehouse: Scale up (expensive)
            Delta: Scale out (S3 infinite)
            
            Growth: No limits, pay-as-you-go
        """
    }
}
```

**Casos de Uso Reales:**

```python
# 1. Databricks - Inventor de Delta Lake
"""
Cliente: Comcast (Telecomunicaciones)
Datos: 2 PB diarios (network logs, streaming video, IoT)
Arquitectura anterior: Lambda (Hadoop batch + Kafka streaming)

Migration a Delta:
- 2018: Pilot con 10 TB (1 mes)
- 2019: Migration 500 TB (6 meses)
- 2020: Full adoption 2 PB/día

Resultados:
- Cost: -30% ($2M/año savings)
- Latency: 4h → 15 min para dashboards
- Code: -50% (unified batch+stream)
- Incidents: -60% (ACID transactions)

Stack:
- Ingestion: Spark Streaming
- Storage: Delta Lake en S3
- Compute: Databricks clusters
- Consumption: Tableau + ML models
"""

# 2. Riot Games (League of Legends)
"""
Caso: Game analytics + Player behavior
Datos: 100M+ games/day, petabytes de events
Latency: <5 min para anti-cheat, <1 hour para balance

Delta Architecture:
┌──────────────┐
│ Game Servers │ → Kafka (1M events/s)
└──────────────┘
       ↓
┌──────────────┐
│ Spark Stream │ → Delta Bronze (raw events)
└──────────────┘
       ↓
┌──────────────┐
│ Enrichment   │ → Delta Silver (validated)
└──────────────┘
       ↓
    ┌──┴──┐
    │     │
    ▼     ▼
┌────┐  ┌────┐
│ ML │  │ BI │ → Delta Gold (aggregations)
└────┘  └────┘

Benefits:
- Unified: Game designers query same data ML uses
- Time travel: "Replay match from 2 days ago"
- Schema evolution: Add features without downtime
"""

# 3. Adobe Experience Platform
"""
Challenge: 100s of tenants, isolated data, compliance
Solution: Delta Lake multi-tenant

Architecture:
/delta/tenant_001/events/
/delta/tenant_002/events/
...
/delta/tenant_500/events/

Features:
- Row-level security (GDPR compliance)
- Tenant isolation (Delta sharing)
- Unified operations (single platform)
- Time travel (data recovery per tenant)

Scale:
- 500+ tenants
- 10 PB total data
- 100K writes/s
- <100ms p99 latency
"""
```

**Migración Lambda → Delta:**

```python
migration_strategy = {
    "Phase 1: Setup (Month 1-2)": """
        1. Provision Delta Lake infrastructure
           - S3 bucket con versioning
           - Unity Catalog setup
           - Spark clusters (Databricks/EMR)
        
        2. Crear medallion structure
           /delta/bronze/
           /delta/silver/
           /delta/gold/
        
        3. Setup CI/CD pipelines
           - GitHub Actions para deploy
           - Testing framework (pytest)
           - Monitoring (Datadog/Prometheus)
    """,
    
    "Phase 2: Bronze Layer (Month 3-4)": """
        1. Migrate raw ingestion
           Kafka → Delta Bronze (append-only)
           
        2. Dual-write period
           - Write to both Lambda batch + Delta
           - Compare row counts, checksums
           
        3. Backfill histórico
           HDFS/Hive → Delta Bronze
           (1-time copy, then delete HDFS)
    """,
    
    "Phase 3: Silver Layer (Month 5-6)": """
        1. Migrate transformation logic
           Batch Spark jobs → Delta pipelines
           
        2. Unified batch+stream
           - Remove duplicate code
           - Single pipeline para ambos
           
        3. Data quality checks
           Great Expectations integration
    """,
    
    "Phase 4: Gold Layer (Month 7-8)": """
        1. Migrate aggregations
           Speed layer → Delta Gold micro-batches
           
        2. Decommission serving layer
           Druid/Cassandra → query Delta directly
           
        3. BI tools reconfigure
           Point Tableau/Power BI to Delta
    """,
    
    "Phase 5: Validation (Month 9)": """
        1. Parallel run
           - Lambda still running (backup)
           - Delta serving production traffic
           
        2. Compare metrics
           - Latency, accuracy, cost
           - User feedback (BI teams)
        
        3. Fix edge cases
    """,
    
    "Phase 6: Cutover (Month 10)": """
        1. Decommission Lambda
           - Stop batch jobs
           - Stop speed layer
           - Delete Druid/Cassandra
        
        2. Cleanup
           - Delete HDFS clusters
           - Reclaim savings
        
        3. Training
           - Team training on Delta
           - Documentation update
    """,
    
    "Total": """
        Duration: 10 months
        Cost: $300K engineering + $100K infra
        Ongoing savings: $50K/mes
        ROI: 8 months
    """
}
```

**Best Practices:**

```python
best_practices = {
    "1. Partition Strategy": """
        ✅ Date partitioning (daily/hourly)
        partitionBy("date")
        
        ⚠️ Z-ordering para high-cardinality
        OPTIMIZE events ZORDER BY (user_id, product_id)
        
        ❌ Over-partitioning
        partitionBy("date", "hour", "user_id")  # Millones de particiones!
    """,
    
    "2. Compaction": """
        Auto-compact para streaming:
        .option("autoCompact", "true")
        
        Scheduled OPTIMIZE:
        OPTIMIZE events WHERE date >= current_date() - 7
        
        VACUUM para cleanup:
        VACUUM events RETAIN 168 HOURS  # 7 días
    """,
    
    "3. Schema Evolution": """
        Always mergeSchema para backward compatibility:
        .option("mergeSchema", "true")
        
        Schema validation:
        df.write.format("delta")
          .option("enforceSchema", "true")
          .save()
    """,
    
    "4. Monitoring": """
        Metrics to track:
        - Write throughput (records/s)
        - Read latency (p50, p95, p99)
        - File count (small files problem)
        - Version count (retention policy)
        - Transaction log size
        
        Alerts:
        - File count >100K per partition
        - Transaction log >10 MB
        - Write latency p99 >5s
    """
}
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [3]:
# 🔄 Simulación Práctica: Arquitectura Delta (Lakehouse)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from pathlib import Path
import json

print("=" * 80)
print("🏛️ ARQUITECTURA DELTA (LAKEHOUSE) - UNIFIED BATCH + STREAMING")
print("=" * 80)

# ============= SETUP LAKEHOUSE =============
base_path = Path("outputs/delta_lakehouse_demo")
base_path.mkdir(parents=True, exist_ok=True)

# Simular Delta Lake con Parquet + transaction log
delta_table_path = base_path / "events_delta"
delta_table_path.mkdir(exist_ok=True)
transaction_log_path = delta_table_path / "_delta_log"
transaction_log_path.mkdir(exist_ok=True)

print(f"\n📁 Lakehouse inicializado en: {delta_table_path}")
print(f"   • Data layer: Parquet files")
print(f"   • Metadata layer: Transaction log (ACID)")

# ============= BATCH INGESTION (Histórico) =============
print("\n" + "=" * 80)
print("1️⃣ BATCH INGESTION - Cargar histórico")
print("=" * 80)

np.random.seed(42)
now = datetime.now()

# Generar datos históricos (30 días)
historical_events = []
for i in range(20000):
    timestamp = now - timedelta(days=np.random.randint(0, 30),
                                 hours=np.random.randint(0, 24))
    historical_events.append({
        'event_id': f'E{i:06d}',
        'timestamp': timestamp,
        'user_id': f'U{np.random.randint(1, 1000):04d}',
        'event_type': np.random.choice(['view', 'click', 'purchase', 'cart_add']),
        'amount': np.random.uniform(10, 500) if np.random.random() > 0.7 else 0
    })

df_batch = pd.DataFrame(historical_events)

start_time = time.time()

# Escribir como Parquet particionado (simulando Delta)
df_batch['date'] = df_batch['timestamp'].dt.date
batch_file = delta_table_path / "batch_v0.parquet"
df_batch.to_parquet(batch_file, compression='snappy', index=False)

# Transaction log entry
transaction = {
    'version': 0,
    'timestamp': now.isoformat(),
    'operation': 'WRITE',
    'operationMetrics': {
        'numFiles': 1,
        'numOutputRows': len(df_batch),
        'numOutputBytes': batch_file.stat().st_size
    }
}
with open(transaction_log_path / "00000000000000000000.json", 'w') as f:
    json.dump(transaction, f, indent=2)

batch_time = time.time() - start_time

print(f"✅ Batch write completado en {batch_time:.2f}s")
print(f"📊 Registros escritos: {len(df_batch):,}")
print(f"💾 Tamaño: {batch_file.stat().st_size / 1024 / 1024:.2f} MB")
print(f"📅 Rango temporal: {df_batch['date'].min()} → {df_batch['date'].max()}")
print(f"🔖 Version: 0 (initial load)")

# ============= STREAMING INGESTION (Micro-batches) =============
print("\n" + "=" * 80)
print("2️⃣ STREAMING INGESTION - Micro-batches cada 5 segundos")
print("=" * 80)

# Simular 3 micro-batches de streaming
for batch_num in range(1, 4):
    print(f"\n⚡ Micro-batch #{batch_num}:")
    
    # Generar nuevos eventos
    new_events = []
    for i in range(100):
        new_events.append({
            'event_id': f'S{batch_num}{i:04d}',
            'timestamp': now + timedelta(seconds=batch_num * 5),
            'user_id': f'U{np.random.randint(1, 1000):04d}',
            'event_type': np.random.choice(['view', 'click', 'purchase', 'cart_add']),
            'amount': np.random.uniform(10, 500) if np.random.random() > 0.7 else 0
        })
    
    df_stream = pd.DataFrame(new_events)
    df_stream['date'] = df_stream['timestamp'].dt.date
    
    start_time = time.time()
    
    # Escribir micro-batch (ACID transaction)
    stream_file = delta_table_path / f"stream_v{batch_num}.parquet"
    df_stream.to_parquet(stream_file, compression='snappy', index=False)
    
    # Transaction log
    transaction = {
        'version': batch_num,
        'timestamp': (now + timedelta(seconds=batch_num * 5)).isoformat(),
        'operation': 'STREAMING_UPDATE',
        'operationMetrics': {
            'numFiles': 1,
            'numOutputRows': len(df_stream),
            'numOutputBytes': stream_file.stat().st_size
        }
    }
    with open(transaction_log_path / f"{batch_num:020d}.json", 'w') as f:
        json.dump(transaction, f, indent=2)
    
    stream_time = time.time() - start_time
    
    print(f"   ✅ Escrito en {stream_time*1000:.2f}ms")
    print(f"   📊 Registros: {len(df_stream)}")
    print(f"   🔖 Version: {batch_num}")

# ============= UNIFIED QUERY (Time Travel) =============
print("\n" + "=" * 80)
print("3️⃣ UNIFIED QUERY - Lectura con Time Travel")
print("=" * 80)

# Leer versión específica (snapshot isolation)
print("\n📸 Snapshot Query: Leer version 0 (solo batch histórico)")
df_v0 = pd.read_parquet(delta_table_path / "batch_v0.parquet")
print(f"   Registros: {len(df_v0):,}")
print(f"   Revenue: ${df_v0['amount'].sum():,.2f}")

print("\n📸 Snapshot Query: Leer version 3 (batch + todos los streams)")
all_files = list(delta_table_path.glob("*.parquet"))
df_current = pd.concat([pd.read_parquet(f) for f in all_files], ignore_index=True)
print(f"   Registros: {len(df_current):,}")
print(f"   Revenue: ${df_current['amount'].sum():,.2f}")
print(f"   Incremento: +{len(df_current) - len(df_v0):,} registros")

# ============= ACID GUARANTEES =============
print("\n" + "=" * 80)
print("4️⃣ ACID GUARANTEES - Transacciones")
print("=" * 80)

print("\n🔒 Transaction Log (linearizable history):")
for log_file in sorted(transaction_log_path.glob("*.json")):
    with open(log_file) as f:
        tx = json.load(f)
    print(f"\n   Version {tx['version']}: {tx['operation']}")
    print(f"   → {tx['operationMetrics']['numOutputRows']:,} rows, "
          f"{tx['operationMetrics']['numOutputBytes'] / 1024:.2f} KB")

# ============= COMPARACIÓN CON LAMBDA/KAPPA =============
print("\n" + "=" * 80)
print("📊 DELTA vs LAMBDA vs KAPPA")
print("=" * 80)

comparison = pd.DataFrame([
    {
        'Característica': 'Código',
        'Lambda': '2 pipelines',
        'Kappa': '1 pipeline stream',
        'Delta': '1 pipeline unificado',
        'Mejor': 'Delta/Kappa'
    },
    {
        'Característica': 'Storage',
        'Lambda': 'S3 + Kafka',
        'Kappa': 'Kafka (costoso)',
        'Delta': 'S3/ADLS (barato)',
        'Mejor': 'Delta'
    },
    {
        'Característica': 'ACID',
        'Lambda': '❌ No',
        'Kappa': '❌ No',
        'Delta': '✅ Sí',
        'Mejor': 'Delta'
    },
    {
        'Característica': 'Time Travel',
        'Lambda': '❌ No',
        'Kappa': 'Solo Kafka retention',
        'Delta': '✅ Infinito',
        'Mejor': 'Delta'
    },
    {
        'Característica': 'Latencia',
        'Lambda': 'Speed: <1s',
        'Kappa': '<1s',
        'Delta': '5-30s (micro-batch)',
        'Mejor': 'Kappa'
    },
    {
        'Característica': 'Batch queries',
        'Lambda': '✅ Optimizado',
        'Kappa': '⚠️ Lento',
        'Delta': '✅ Optimizado',
        'Mejor': 'Lambda/Delta'
    }
])

print(comparison.to_string(index=False))

# ============= CASOS DE USO IDEALES =============
print("\n" + "=" * 80)
print("🎯 CUÁNDO USAR DELTA")
print("=" * 80)

use_cases = {
    '✅ Ideal': [
        'Lakehouse unificado (data warehouse + lake)',
        'GDPR/Compliance (auditabilidad, time travel)',
        'Batch + Streaming con misma lógica',
        'Latencia <30s aceptable',
        'Data science + analytics'
    ],
    '⚠️ Evitar': [
        'Latencia <1s crítica (usar Kappa)',
        'Streaming puro sin batch (usar Kappa)',
        'Legacy con Lambda existente (migración costosa)'
    ]
}

for category, items in use_cases.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  • {item}")

print(f"\n" + "=" * 80)
print("✅ Simulación Delta completada")
print(f"   • Batch histórico: {len(df_batch):,} eventos")
print(f"   • Streaming: 3 micro-batches, 300 eventos")
print(f"   • Total: {len(df_current):,} eventos")
print(f"   • Transaction log: 4 versiones con ACID")
print(f"   • Storage: {sum(f.stat().st_size for f in all_files) / 1024 / 1024:.2f} MB")
print("=" * 80)

🏛️ ARQUITECTURA DELTA (LAKEHOUSE) - UNIFIED BATCH + STREAMING

📁 Lakehouse inicializado en: outputs\delta_lakehouse_demo\events_delta
   • Data layer: Parquet files
   • Metadata layer: Transaction log (ACID)

1️⃣ BATCH INGESTION - Cargar histórico
✅ Batch write completado en 0.08s
📊 Registros escritos: 20,000
💾 Tamaño: 0.27 MB
📅 Rango temporal: 2025-11-09 → 2025-12-09
🔖 Version: 0 (initial load)

2️⃣ STREAMING INGESTION - Micro-batches cada 5 segundos

⚡ Micro-batch #1:
   ✅ Escrito en 5.53ms
   📊 Registros: 100
   🔖 Version: 1

⚡ Micro-batch #2:
   ✅ Escrito en 3.01ms
   📊 Registros: 100
   🔖 Version: 2

⚡ Micro-batch #3:
   ✅ Escrito en 4.80ms
   📊 Registros: 100
   🔖 Version: 3

3️⃣ UNIFIED QUERY - Lectura con Time Travel

📸 Snapshot Query: Leer version 0 (solo batch histórico)
   Registros: 20,000
   Revenue: $1,516,429.23

📸 Snapshot Query: Leer version 3 (batch + todos los streams)
   Registros: 20,300
   Revenue: $1,539,993.67
   Incremento: +300 registros

4️⃣ ACID GUARANTEES 

### 🌐 **Data Mesh: Paradigma Organizacional Descentralizado**

**Problema: Data Platform Centralizado (Monolito)**

```
Organización tradicional:
┌────────────────────────────────────────────────┐
│         CENTRAL DATA TEAM (20 personas)        │
│                                                 │
│  Responsable de:                                │
│  • Todos los pipelines                         │
│  • Todos los data models                       │
│  • Todos los dashboards                        │
│  • Todos los ML models                         │
└────────────┬───────────────────────────────────┘
             │
   ┌─────────┴─────────┐
   │  DATA WAREHOUSE   │
   │   (Single DB)     │
   └─────────┬─────────┘
             │
    ┌────────┴────────┐
    │                 │
    ▼                 ▼
┌────────┐      ┌──────────┐
│ Ventas │      │Logística │ ... (10+ dominios)
│ Team   │      │ Team     │
└────────┘      └──────────┘
   ↓                 ↓
Request            Request
pipeline          pipeline
   ↓                 ↓
  ⏰ 3 meses        ⏰ 3 meses
  🐛 Bugs           🐛 Bugs
  📉 Low quality    📉 Low quality

Problemas:
❌ Bottleneck: Central team sobrecargado (backlog 6+ meses)
❌ Context loss: Data team no entiende dominio de negocio
❌ Scalability: Imposible crecer linearmente
❌ Ownership: Nadie responsable de calidad de datos
```

**Solución: Data Mesh (Zhamak Dehghani, 2019)**

```
Data Mesh: Descentralización por dominio
┌─────────────────────────────────────────────┐
│        FEDERATED GOVERNANCE                  │
│  (Políticas centrales, enforcement local)    │
└────┬──────────────┬──────────────┬───────────┘
     │              │              │
┌────▼─────┐  ┌────▼─────┐  ┌────▼─────┐
│ VENTAS   │  │LOGÍSTICA │  │ FINANZAS │
│ DOMAIN   │  │ DOMAIN   │  │ DOMAIN   │
│          │  │          │  │          │
│ Data     │  │ Data     │  │ Data     │
│ Product  │  │ Product  │  │ Product  │
│ Owner    │  │ Owner    │  │ Owner    │
│          │  │          │  │          │
│ • ETL    │  │ • ETL    │  │ • ETL    │
│ • Quality│  │ • Quality│  │ • Quality│
│ • API    │  │ • API    │  │ • API    │
│ • Docs   │  │ • Docs   │  │ • Docs   │
└────┬─────┘  └────┬─────┘  └────┬─────┘
     │              │              │
     └──────────────┴──────────────┘
                    │
         ┌──────────▼──────────┐
         │ SELF-SERVE PLATFORM │
         │ (Infra común)       │
         │ • Spark/Airflow     │
         │ • Monitoring        │
         │ • Data Catalog      │
         └─────────────────────┘
```

**4 Principios Fundamentales:**

```python
principios_data_mesh = {
    "1. Domain-Oriented Decentralization": """
        Datos pertenecen al dominio de negocio
        
        Ejemplo: E-commerce
        ┌────────────────────────────────────────┐
        │ DOMAIN: Ventas                          │
        │ Owner: VP of Sales                      │
        │ Data Products:                          │
        │  • orders (transaccional)               │
        │  • orders_aggregated (analítico)        │
        │  • customer_segments (ML)               │
        │                                         │
        │ Team composition:                       │
        │  • Product Manager (prioridades)        │
        │  • Data Engineer (pipelines)            │
        │  • Analytics Engineer (dbt models)      │
        │  • Data Analyst (consumers)             │
        └────────────────────────────────────────┘
        
        Ventajas:
        ✅ Domain expertise: Equipo entiende negocio
        ✅ Velocity: No esperar central team
        ✅ Accountability: Owner claro de calidad
    """,
    
    "2. Data as a Product": """
        Cada dataset es un producto con:
        
        📋 SLA (Service Level Agreement):
        - Latency: <15 min para orders_aggregated
        - Availability: 99.9% uptime
        - Freshness: Updated cada 5 min
        - Quality: >95% completeness
        
        📖 Documentation:
        - Schema: Columnas, tipos, constraints
        - Lineage: De dónde viene cada campo
        - Examples: Query samples
        - Contact: Slack channel, owner email
        
        🔒 Access Control:
        - Public: Todos pueden leer
        - Private: Solo equipo ventas
        - PII: Masked para no-autorizados
        
        📊 Observability:
        - Metrics: Row count, size, update time
        - Alerts: SLA violations
        - Changelog: Version history
        
        Ejemplo manifest.yaml:
        ```
        product_name: orders_aggregated
        owner: ventas-data@company.com
        sla:
          latency_minutes: 15
          availability_percent: 99.9
          quality_score: 0.95
        schema_url: s3://catalog/ventas/orders_agg/schema.json
        lineage:
          sources:
            - orders_raw
            - customers
            - products
        access:
          read: [sales_team, finance_team, executives]
          write: [sales_data_engineers]
        ```
    """,
    
    "3. Self-Serve Data Platform": """
        Infraestructura común para todos los dominios
        
        Platform Components:
        ┌─────────────────────────────────────┐
        │ COMPUTE                              │
        │  • Spark clusters (on-demand)       │
        │  • Airflow (orchestration)          │
        │  • dbt (transformations)            │
        └─────────────────────────────────────┘
        
        ┌─────────────────────────────────────┐
        │ STORAGE                              │
        │  • Delta Lake (lakehouse)           │
        │  • S3 buckets (per-domain)          │
        │  • Unity Catalog (metadata)         │
        └─────────────────────────────────────┘
        
        ┌─────────────────────────────────────┐
        │ OBSERVABILITY                        │
        │  • Datadog (metrics, logs)          │
        │  • Great Expectations (quality)     │
        │  • OpenLineage (lineage tracking)   │
        └─────────────────────────────────────┘
        
        ┌─────────────────────────────────────┐
        │ GOVERNANCE                           │
        │  • Data Catalog (Amundsen/DataHub)  │
        │  • Access Control (Ranger/Unity)    │
        │  • Policy Engine (OPA)              │
        └─────────────────────────────────────┘
        
        Self-service workflow:
        1. Team crea repo: github.com/company/ventas-data
        2. Define product: manifest.yaml + dbt models
        3. CI/CD deploy: Tests → Stage → Prod
        4. Auto-register: Catalog actualizado
        5. Monitoring: Dashboards auto-generated
        
        Platform team provee:
        - Templates (cookiecutter)
        - Best practices docs
        - On-call support
        - Cost monitoring
    """,
    
    "4. Federated Computational Governance": """
        Políticas globales + Autonomía local
        
        GLOBAL POLICIES (Central governance):
        ┌────────────────────────────────────┐
        │ 1. Security                         │
        │    • PII must be encrypted at rest │
        │    • Access logs retained 1 year   │
        │    • MFA required for production   │
        │                                     │
        │ 2. Compliance (GDPR, CCPA)         │
        │    • Right to be forgotten         │
        │    • Data retention max 7 years    │
        │    • Audit trail required          │
        │                                     │
        │ 3. Quality                          │
        │    • All products have tests       │
        │    • SLA violations reported       │
        │    • Schema changes versioned      │
        │                                     │
        │ 4. Interoperability                │
        │    • Standard formats (Parquet)    │
        │    • Common IDs (user_id format)   │
        │    • Shared ontology (terms)       │
        └────────────────────────────────────┘
        
        LOCAL AUTONOMY (Domain teams):
        ┌────────────────────────────────────┐
        │ • Choose transformation tools       │
        │   (dbt, Spark, Python)             │
        │ • Define data models               │
        │ • Set refresh frequency            │
        │ • Manage access within domain      │
        │ • Optimize costs                   │
        └────────────────────────────────────┘
        
        Enforcement:
        - Automated checks in CI/CD
        - Policy-as-code (OPA policies)
        - Failing checks block deploy
        - Dashboards show compliance
    """
}
```

**Implementación Real: Data Mesh Stack**

```python
# Estructura organizacional
data_mesh_org = {
    "Domain Teams (Autónomos)": {
        "Ventas Domain": {
            "Members": "8 personas",
            "Products": [
                "orders (transactional)",
                "orders_daily_agg (analytical)",
                "customer_rfm_segments (ML)"
            ],
            "Tech Stack": "Spark, dbt, Delta Lake",
            "Budget": "$50K/mes"
        },
        
        "Logística Domain": {
            "Members": "6 personas",
            "Products": [
                "shipments (operational)",
                "delivery_performance (metrics)",
                "route_optimization (ML)"
            ],
            "Tech Stack": "Flink, Kafka, Iceberg",
            "Budget": "$40K/mes"
        },
        
        "Marketing Domain": {
            "Members": "5 personas",
            "Products": [
                "campaigns (metadata)",
                "attribution (analytics)",
                "propensity_scores (ML)"
            ],
            "Tech Stack": "Airflow, Python, Delta Lake",
            "Budget": "$30K/mes"
        }
    },
    
    "Platform Team (Enabling)": {
        "Members": "10 personas",
        "Responsibilities": [
            "Maintain Spark/Airflow infrastructure",
            "Operate Data Catalog",
            "Enforce governance policies",
            "Cost optimization",
            "Training & documentation"
        ],
        "Budget": "$100K/mes",
        "Ratio": "1 platform engineer per 20 domain engineers"
    },
    
    "Governance Team (Federated)": {
        "Members": "3 personas",
        "Responsibilities": [
            "Define global policies",
            "Compliance (GDPR, SOX)",
            "Audit & reporting",
            "Cross-domain standards"
        ],
        "Budget": "$20K/mes"
    }
}

# Data Product Example
data_product_spec = '''
# orders_daily_agg Data Product

## Overview
Daily aggregated orders for BI dashboards and reporting.

## Owner
- Team: Ventas Data Engineering
- Contact: ventas-data@company.com
- Slack: #ventas-data

## SLA
- Freshness: Updated daily @6AM UTC
- Latency: <30 minutes processing time
- Availability: 99.9% (max 8.76h downtime/year)
- Quality: >99% completeness, >98% accuracy

## Schema
| Column | Type | Description | PII |
|--------|------|-------------|-----|
| date | DATE | Order date | No |
| country | STRING | Country code (ISO) | No |
| total_orders | BIGINT | Count of orders | No |
| total_revenue | DECIMAL(10,2) | Sum of revenue (USD) | No |
| avg_order_value | DECIMAL(10,2) | Average order value | No |

## Lineage
```
orders_raw (Bronze)
    ↓
orders_validated (Silver)
    ↓
orders_daily_agg (Gold) ← THIS PRODUCT
```

## Access
- Read: sales_team, finance_team, executives
- Write: sales_data_engineers
- Admin: sales_domain_owner

## Usage Examples
```sql
-- Get last 7 days
SELECT * FROM orders_daily_agg
WHERE date >= CURRENT_DATE - INTERVAL 7 DAYS
ORDER BY date DESC

-- YoY comparison
SELECT 
    date,
    total_revenue,
    LAG(total_revenue, 365) OVER (ORDER BY date) as prev_year_revenue
FROM orders_daily_agg
WHERE date >= '2024-01-01'
```

## Metrics
- Dashboard: https://monitoring.company.com/ventas/orders_agg
- Alerts: PagerDuty team "sales-data-oncall"

## Changelog
- 2025-10-01: Added avg_order_value column
- 2025-09-15: Changed country to ISO codes
- 2025-08-01: Initial release
'''
```

**Ventajas de Data Mesh:**

```python
ventajas_mesh = {
    "1. Scalability": """
        Tradicional: Central team → linear growth impossible
        10 domains × 1 central team = bottleneck
        
        Data Mesh: Domain teams → linear scaling
        10 domains × 10 teams = 100 engineers productive
        
        Growth: Agregar dominio no afecta otros
    """,
    
    "2. Domain Expertise": """
        Central team:
        "¿Qué significa 'adjusted_revenue'?"
        → Ask business (delays)
        
        Domain team:
        Engineers trabajan con business daily
        → Deep understanding, mejor calidad
    """,
    
    "3. Velocity": """
        Backlog central: 6 meses
        Domain ownership: 2 semanas
        
        Feature request → deploy:
        Central: 3-6 meses
        Mesh: 1-2 sprints
    """,
    
    "4. Quality": """
        Central team: Best effort (no ownership)
        Domain team: Accountable (SLAs, metrics)
        
        Data quality:
        Central: 85% average
        Mesh: 95% average (incentivized)
    """
}
```

**Desafíos de Data Mesh:**

```python
desafios_mesh = {
    "1. Organizational Maturity": """
        Requiere:
        - DevOps culture (CI/CD, testing)
        - Data literacy en equipos de negocio
        - Executive buy-in (budget descentralizado)
        
        NO funciona si:
        - Organización muy jerárquica
        - Equipos no autónomos
        - Sin cultura de ownership
        
        Ejemplo fracaso:
        Company X intentó mesh sin DevOps
        → Dominios no saben deploy pipelines
        → Rollback a central team en 6 meses
    """,
    
    "2. Platform Complexity": """
        Self-serve platform = sophisticated
        
        Requiere:
        - Multi-tenancy (aislamiento dominios)
        - Cost tracking per domain
        - Automated provisioning
        - Standardized observability
        
        Platform team: 10-15 personas mínimo
        Cost: $1M+/año (salaries + infra)
        
        Break-even: ~100+ data engineers total
    """,
    
    "3. Duplication Risk": """
        Sin governance fuerte:
        - 3 equipos crean "customer_dim" diferente
        - Métricas inconsistentes cross-domain
        - Redundant data copies (cost)
        
        Mitigación:
        - Data Catalog mandatory
        - Shared domain for common entities
        - Regular cross-domain sync meetings
    """,
    
    "4. Skillset Gap": """
        Domain engineers necesitan:
        - Data engineering (ETL)
        - Analytics engineering (dbt)
        - DevOps (CI/CD)
        - Domain knowledge (business)
        
        Hiring difficult: Full-stack data engineers
        Training: 6-12 meses ramp-up
        
        Alternative: Embedded specialists
        - 1 data engineer per 2 domains
        - Rotates between teams
    """
}
```

**Casos de Uso (Cuándo Data Mesh Funciona):**

```python
# ✅ 1. LARGE ORG con múltiples dominios
"""
Empresa: Zalando (E-commerce Europeo)
Tamaño: 10,000+ empleados, 20+ países
Dominios: 50+ (Fashion, Logistics, Payments, ...)

Data Mesh adoption (2020):
- 50 domain teams autónomos
- Platform team: 25 personas
- 200+ data products published
- Self-serve: 90% requests no need central

Results:
- Time-to-market: 6 meses → 2 semanas
- Data quality: 80% → 95%
- Team satisfaction: +40%
"""

# ✅ 2. DISTRIBUTED company
"""
Empresa: Confluent (remote-first)
Dominios: Customer Success, Engineering, Sales
Challenge: Central team = timezone hell

Data Mesh benefits:
- Teams own data in their timezone
- Async collaboration (via catalog)
- No central bottleneck
"""

# ❌ 3. SMALL STARTUP (<100 personas)
"""
Mesh overhead > benefits
Better: Small central team (5 personas)
Mesh when: Reach 200+ engineers
"""

# ❌ 4. HIGHLY REGULATED (sin autonomía)
"""
Banking: Regulators require central control
Mesh autonomy conflicts with compliance
Better: Centralized with strong governance
"""
```

**Migration Path: Central → Data Mesh**

```python
migration_roadmap = {
    "Year 1: Foundation": """
        Q1: Executive alignment
        - Present mesh vision
        - Secure budget
        - Identify pilot domain
        
        Q2: Platform MVP
        - Self-serve Spark/Airflow
        - Data Catalog (Amundsen)
        - Basic CI/CD templates
        
        Q3-Q4: Pilot domain
        - Choose 1 domain (e.g., Sales)
        - Migrate 3 products
        - Learn lessons, iterate
    """,
    
    "Year 2: Scale": """
        Q1-Q2: Onboard 5 domains
        - Training programs
        - Platform improvements
        - Governance policies drafted
        
        Q3-Q4: Federated governance
        - Policy engine (OPA)
        - Compliance automation
        - Cross-domain standards
    """,
    
    "Year 3: Maturity": """
        Q1-Q2: Onboard remaining domains
        - 100% domains on mesh
        - Decommission central team legacy
        
        Q3-Q4: Optimization
        - Cost optimization
        - Advanced features (ML platform)
        - Community of practice
    """,
    
    "Total Investment": """
        Platform team: 10 personas × $150K × 3 años = $4.5M
        Training: $500K
        Tooling: $1M
        Total: $6M
        
        Break-even: Year 4 (velocity gains)
    """
}
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [4]:
# 🔄 Simulación Práctica: Data Mesh - Arquitectura Organizacional
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
from pathlib import Path

print("=" * 80)
print("🌐 DATA MESH - ARQUITECTURA DESCENTRALIZADA POR DOMINIOS")
print("=" * 80)

# ============= SETUP ORGANIZATION =============
np.random.seed(42)
now = datetime.now()

mesh_base = Path("outputs/data_mesh_demo")
mesh_base.mkdir(parents=True, exist_ok=True)

print("\n🏢 Organización: E-commerce Multilatino")
print("   • 3 dominios de negocio con ownership descentralizado")
print("   • Plataforma self-service compartida")
print("   • Gobernanza federada")

# ============= DOMINIO 1: VENTAS =============
print("\n" + "=" * 80)
print("🛒 DOMINIO 1: VENTAS (Sales Domain)")
print("=" * 80)

# Data Product: Ventas Transaccionales
sales_data = []
for i in range(5000):
    sales_data.append({
        'transaction_id': f'T{i:06d}',
        'timestamp': now - timedelta(days=np.random.randint(0, 30)),
        'customer_id': f'C{np.random.randint(1, 500):04d}',
        'product_id': f'P{np.random.randint(1, 100):03d}',
        'amount': np.random.uniform(10, 500),
        'channel': np.random.choice(['web', 'mobile', 'store'])
    })

df_sales = pd.DataFrame(sales_data)

# Publicar Data Product
sales_product_path = mesh_base / "sales_domain"
sales_product_path.mkdir(exist_ok=True)
df_sales.to_parquet(sales_product_path / "transactions.parquet", index=False)

# Data Product Contract
sales_contract = {
    'name': 'sales.transactions',
    'owner': 'sales-team@company.com',
    'version': '2.1.0',
    'slo': {
        'freshness': '<15 min',
        'availability': '99.9%',
        'quality': 'completeness > 99%'
    },
    'schema': {
        'transaction_id': 'string (unique)',
        'timestamp': 'datetime',
        'customer_id': 'string (FK to customers)',
        'product_id': 'string (FK to products)',
        'amount': 'decimal',
        'channel': 'enum[web, mobile, store]'
    },
    'governance': {
        'pii_fields': ['customer_id'],
        'retention': '7 years',
        'access': 'all authenticated users'
    }
}

with open(sales_product_path / "contract.json", 'w') as f:
    json.dump(sales_contract, f, indent=2)

print(f"✅ Data Product publicado: {sales_contract['name']}")
print(f"   Owner: {sales_contract['owner']}")
print(f"   SLO Freshness: {sales_contract['slo']['freshness']}")
print(f"   Registros: {len(df_sales):,}")
print(f"   Revenue total: ${df_sales['amount'].sum():,.2f}")

# ============= DOMINIO 2: LOGÍSTICA =============
print("\n" + "=" * 80)
print("📦 DOMINIO 2: LOGÍSTICA (Logistics Domain)")
print("=" * 80)

# Data Product: Envíos y Entregas
logistics_data = []
for i in range(4500):
    logistics_data.append({
        'shipment_id': f'S{i:06d}',
        'transaction_id': df_sales['transaction_id'].sample(1).iloc[0],  # FK
        'created_at': now - timedelta(days=np.random.randint(0, 30)),
        'delivered_at': now - timedelta(days=np.random.randint(0, 25)) if np.random.random() > 0.2 else None,
        'status': np.random.choice(['pending', 'in_transit', 'delivered', 'returned']),
        'warehouse': np.random.choice(['MX-CDMX', 'CO-BOG', 'AR-BUE', 'CL-SCL'])
    })

df_logistics = pd.DataFrame(logistics_data)

# Publicar Data Product
logistics_product_path = mesh_base / "logistics_domain"
logistics_product_path.mkdir(exist_ok=True)
df_logistics.to_parquet(logistics_product_path / "shipments.parquet", index=False)

# Data Product Contract
logistics_contract = {
    'name': 'logistics.shipments',
    'owner': 'logistics-team@company.com',
    'version': '1.8.2',
    'slo': {
        'freshness': '<5 min',
        'availability': '99.95%',
        'quality': 'no orphan shipments (FK integrity)'
    },
    'schema': {
        'shipment_id': 'string (unique)',
        'transaction_id': 'string (FK to sales.transactions)',
        'created_at': 'datetime',
        'delivered_at': 'datetime (nullable)',
        'status': 'enum[pending, in_transit, delivered, returned]',
        'warehouse': 'string'
    },
    'governance': {
        'pii_fields': [],
        'retention': '3 years',
        'access': 'logistics + analytics teams'
    }
}

with open(logistics_product_path / "contract.json", 'w') as f:
    json.dump(logistics_contract, f, indent=2)

print(f"✅ Data Product publicado: {logistics_contract['name']}")
print(f"   Owner: {logistics_contract['owner']}")
print(f"   SLO Freshness: {logistics_contract['slo']['freshness']}")
print(f"   Registros: {len(df_logistics):,}")
print(f"   Entregas completadas: {df_logistics['status'].value_counts().get('delivered', 0):,}")

# ============= DOMINIO 3: FINANZAS =============
print("\n" + "=" * 80)
print("💰 DOMINIO 3: FINANZAS (Finance Domain)")
print("=" * 80)

# Data Product: Pagos y Facturación
finance_data = []
for i in range(4800):
    amount = np.random.uniform(10, 500)
    finance_data.append({
        'payment_id': f'PAY{i:06d}',
        'transaction_id': df_sales['transaction_id'].sample(1).iloc[0],  # FK
        'payment_date': now - timedelta(days=np.random.randint(0, 30)),
        'amount': amount,
        'fee': amount * 0.03,  # 3% payment processing fee
        'method': np.random.choice(['credit_card', 'debit', 'transfer', 'cash']),
        'status': np.random.choice(['approved', 'pending', 'declined'], p=[0.85, 0.10, 0.05])
    })

df_finance = pd.DataFrame(finance_data)

# Publicar Data Product
finance_product_path = mesh_base / "finance_domain"
finance_product_path.mkdir(exist_ok=True)
df_finance.to_parquet(finance_product_path / "payments.parquet", index=False)

# Data Product Contract
finance_contract = {
    'name': 'finance.payments',
    'owner': 'finance-team@company.com',
    'version': '3.0.1',
    'slo': {
        'freshness': '<1 min (real-time)',
        'availability': '99.99%',
        'quality': 'amount = sales.amount (reconciliation)'
    },
    'schema': {
        'payment_id': 'string (unique)',
        'transaction_id': 'string (FK to sales.transactions)',
        'payment_date': 'datetime',
        'amount': 'decimal',
        'fee': 'decimal',
        'method': 'enum[credit_card, debit, transfer, cash]',
        'status': 'enum[approved, pending, declined]'
    },
    'governance': {
        'pii_fields': ['payment_id'],
        'retention': '10 years (compliance)',
        'access': 'finance team only (PCI-DSS)'
    }
}

with open(finance_product_path / "contract.json", 'w') as f:
    json.dump(finance_contract, f, indent=2)

print(f"✅ Data Product publicado: {finance_contract['name']}")
print(f"   Owner: {finance_contract['owner']}")
print(f"   SLO Freshness: {finance_contract['slo']['freshness']}")
print(f"   Registros: {len(df_finance):,}")
print(f"   Pagos aprobados: ${df_finance[df_finance['status']=='approved']['amount'].sum():,.2f}")

# ============= CONSUMER: ANALYTICS DOMAIN =============
print("\n" + "=" * 80)
print("📊 CONSUMER: ANALYTICS DOMAIN (Cross-Domain Query)")
print("=" * 80)

print("\n🔍 Caso: Análisis de conversión completo (Sales → Logistics → Finance)")

# Leer todos los Data Products publicados
df_sales_consumed = pd.read_parquet(sales_product_path / "transactions.parquet")
df_logistics_consumed = pd.read_parquet(logistics_product_path / "shipments.parquet")
df_finance_consumed = pd.read_parquet(finance_product_path / "payments.parquet")

# Join cross-domain
df_full = df_sales_consumed.merge(
    df_logistics_consumed, on='transaction_id', how='left'
).merge(
    df_finance_consumed, on='transaction_id', how='left'
)

print(f"\n📈 Métricas del Funnel:")
print(f"   Ventas iniciadas: {len(df_sales_consumed):,}")
print(f"   Envíos creados: {df_logistics_consumed['shipment_id'].notna().sum():,} ({len(df_logistics_consumed)/len(df_sales_consumed)*100:.1f}%)")
print(f"   Pagos aprobados: {df_finance_consumed[df_finance_consumed['status']=='approved'].shape[0]:,} ({df_finance_consumed[df_finance_consumed['status']=='approved'].shape[0]/len(df_sales_consumed)*100:.1f}%)")

delivered = df_full[df_full['status_y'] == 'delivered']
print(f"   Entregas completadas: {len(delivered):,} ({len(delivered)/len(df_sales_consumed)*100:.1f}%)")

# ============= GOBERNANZA FEDERADA =============
print("\n" + "=" * 80)
print("🔐 GOBERNANZA FEDERADA")
print("=" * 80)

governance_summary = pd.DataFrame([
    {
        'Domain': 'Sales',
        'Data Product': 'transactions',
        'Owner': 'sales-team',
        'PII': 'Yes (customer_id)',
        'Retention': '7 years',
        'SLO Freshness': '<15 min'
    },
    {
        'Domain': 'Logistics',
        'Data Product': 'shipments',
        'Owner': 'logistics-team',
        'PII': 'No',
        'Retention': '3 years',
        'SLO Freshness': '<5 min'
    },
    {
        'Domain': 'Finance',
        'Data Product': 'payments',
        'Owner': 'finance-team',
        'PII': 'Yes (payment_id)',
        'Retention': '10 years',
        'SLO Freshness': '<1 min'
    }
])

print(governance_summary.to_string(index=False))

# ============= PRINCIPIOS DATA MESH =============
print("\n" + "=" * 80)
print("🧭 PRINCIPIOS DE DATA MESH APLICADOS")
print("=" * 80)

principles = {
    '1. Domain-Oriented Decentralization': [
        f'✅ 3 dominios con ownership claro',
        f'✅ Cada equipo publica sus propios Data Products',
        f'✅ No hay equipo central de datos'
    ],
    '2. Data as a Product': [
        f'✅ Contratos versionados (sales v2.1.0, logistics v1.8.2, finance v3.0.1)',
        f'✅ SLOs definidos (freshness, availability, quality)',
        f'✅ Documentación completa (schema, governance)'
    ],
    '3. Self-Serve Platform': [
        f'✅ Storage común (Parquet en file system)',
        f'✅ Formato estándar (Parquet + JSON contracts)',
        f'✅ Discovery automático (contracts.json)'
    ],
    '4. Federated Governance': [
        f'✅ Políticas globales (PII marcado, retention definido)',
        f'✅ Autonomía local (cada dominio elige su SLO)',
        f'✅ Compliance distribuido (PCI-DSS en finance, GDPR en sales)'
    ]
}

for principle, items in principles.items():
    print(f"\n{principle}:")
    for item in items:
        print(f"  {item}")

# ============= TRADE-OFFS =============
print("\n" + "=" * 80)
print("⚖️ TRADE-OFFS DE DATA MESH")
print("=" * 80)

trade_offs = pd.DataFrame([
    {'Aspecto': 'Escalabilidad org', 'Monolito': '⚠️ Cuello botella', 'Data Mesh': '✅ Lineal con dominios'},
    {'Aspecto': 'Time to market', 'Monolito': 'Semanas (cola)', 'Data Mesh': 'Días (autonomía)'},
    {'Aspecto': 'Complejidad técnica', 'Monolito': 'Baja (1 pipeline)', 'Data Mesh': '⚠️ Alta (N pipelines)'},
    {'Aspecto': 'Madurez requerida', 'Monolito': 'Junior friendly', 'Data Mesh': '⚠️ Senior+ (DevOps, SRE)'},
    {'Aspecto': 'Costos iniciales', 'Monolito': 'Bajo', 'Data Mesh': '⚠️ Alto (plataforma)'},
])

print(trade_offs.to_string(index=False))

print(f"\n" + "=" * 80)
print("✅ Simulación Data Mesh completada")
print(f"   • 3 dominios publicaron Data Products")
print(f"   • {len(df_sales) + len(df_logistics) + len(df_finance):,} registros totales")
print(f"   • Analytics consumió cross-domain exitosamente")
print(f"   • Gobernanza federada aplicada")
print("=" * 80)

🌐 DATA MESH - ARQUITECTURA DESCENTRALIZADA POR DOMINIOS

🏢 Organización: E-commerce Multilatino
   • 3 dominios de negocio con ownership descentralizado
   • Plataforma self-service compartida
   • Gobernanza federada

🛒 DOMINIO 1: VENTAS (Sales Domain)
✅ Data Product publicado: sales.transactions
   Owner: sales-team@company.com
   SLO Freshness: <15 min
   Registros: 5,000
   Revenue total: $1,269,904.63

📦 DOMINIO 2: LOGÍSTICA (Logistics Domain)
✅ Data Product publicado: logistics.shipments
   Owner: logistics-team@company.com
   SLO Freshness: <5 min
   Registros: 4,500
   Entregas completadas: 1,197

💰 DOMINIO 3: FINANZAS (Finance Domain)
✅ Data Product publicado: logistics.shipments
   Owner: logistics-team@company.com
   SLO Freshness: <5 min
   Registros: 4,500
   Entregas completadas: 1,197

💰 DOMINIO 3: FINANZAS (Finance Domain)
✅ Data Product publicado: finance.payments
   Owner: finance-team@company.com
   SLO Freshness: <1 min (real-time)
   Registros: 4,800
   Pagos aprob

- Batch layer: histórico completo, recalculable, alta latencia (Spark/Hadoop).
- Speed layer: streaming incremental, baja latencia (Kafka/Flink/Spark Streaming).
- Serving layer: merge de vistas batch + speed para consultas (Druid/Cassandra/BigQuery).
- Trade-offs: doble lógica (batch + stream), complejidad operacional, eventual consistency.

In [5]:
lambda_diagram = '''
┌────────────┐
│   Data     │
│  Sources   │
└─────┬──────┘
      │
   ┌──┴───┐
   │Queue │ (Kafka)
   └──┬───┘
      │
 ┌────┴─────┐
 │  Batch   │  (Spark jobs nocturnos)
 │  Layer   │
 └────┬─────┘
      │
   Master
   Dataset
      │
   ┌──┴───┐
   │Serving│ (Druid/BigQuery)
   │ Layer │
   └──────┘
      ↑
   ┌──┴───┐
   │Speed │ (Flink/Spark Streaming)
   │Layer │
   └──────┘
'''
print(lambda_diagram)


┌────────────┐
│   Data     │
│  Sources   │
└─────┬──────┘
      │
   ┌──┴───┐
   │Queue │ (Kafka)
   └──┬───┘
      │
 ┌────┴─────┐
 │  Batch   │  (Spark jobs nocturnos)
 │  Layer   │
 └────┬─────┘
      │
   Master
   Dataset
      │
   ┌──┴───┐
   │Serving│ (Druid/BigQuery)
   │ Layer │
   └──────┘
      ↑
   ┌──┴───┐
   │Speed │ (Flink/Spark Streaming)
   │Layer │
   └──────┘



## 2. Arquitectura Kappa

- Un solo pipeline streaming para todo (batch = replay del log).
- Simplifica operaciones y lógica unificada.
- Requiere log infinito (Kafka con retención larga) y capacidad de reprocessing.
- Trade-offs: coste de almacenamiento del log, complejidad del estado, menor optimización batch.

## 3. Arquitectura Delta (Lakehouse)

- Unifica batch y streaming sobre un único storage transaccional (Delta/Iceberg).
- Streaming escribe micro-batches transaccionales; batch lee histórico con time travel.
- Elimina duplicación de código y simplifica gobernanza (un catálogo, un linaje).
- Trade-offs: requiere adopción de formato y motor compatible (Spark/Trino).

## 4. Data Mesh: paradigma organizacional

- Descentralización de la propiedad de datos por dominio de negocio.
- Data products: APIs/tablas con SLOs, documentación, linaje y contratos.
- Plataforma self-service: herramientas comunes (CI/CD, catálogo, observabilidad).
- Gobernanza federada: políticas globales + autonomía local.
- Trade-offs: requiere madurez organizacional, infraestructura sólida, cambio cultural.

In [6]:
mesh_principles = '''
1. Domain-oriented decentralization: equipos de dominio (ventas, logística, finanzas) son dueños de sus datos.
2. Data as a product: cada dataset tiene owner, SLO, versionado, documentación y calidad garantizada.
3. Self-serve platform: infraestructura común (orquestación, observabilidad, catálogo) sin silos.
4. Federated governance: políticas de seguridad, privacidad y calidad definidas centralmente, aplicadas localmente.
'''
print(mesh_principles)


1. Domain-oriented decentralization: equipos de dominio (ventas, logística, finanzas) son dueños de sus datos.
2. Data as a product: cada dataset tiene owner, SLO, versionado, documentación y calidad garantizada.
3. Self-serve platform: infraestructura común (orquestación, observabilidad, catálogo) sin silos.
4. Federated governance: políticas de seguridad, privacidad y calidad definidas centralmente, aplicadas localmente.



## 5. Comparación y cuándo elegir cada una

In [7]:
import pandas as pd
comp = pd.DataFrame([
  {'Arquitectura':'Lambda', 'Latencia':'Batch: horas, Speed: segundos', 'Complejidad':'Alta (doble lógica)', 'Casos':'Legacy con requerimientos mixtos'},
  {'Arquitectura':'Kappa', 'Latencia':'Baja (streaming)', 'Complejidad':'Media (un pipeline)', 'Casos':'Todo es streaming, reprocessing factible'},
  {'Arquitectura':'Delta', 'Latencia':'Baja-Media (micro-batch)', 'Complejidad':'Media (un storage)', 'Casos':'Lakehouse, unificación batch/stream'},
  {'Arquitectura':'Data Mesh', 'Latencia':'Variable (por dominio)', 'Complejidad':'Alta (organizacional)', 'Casos':'Grandes org con múltiples dominios'}
])
comp

,Arquitectura,Latencia,Complejidad,Casos
0,Lambda,"Batch: horas, Speed: segundos",Alta (doble lógica),Legacy con requerimientos mixtos
1,Kappa,Baja (streaming),Media (un pipeline),"Todo es streaming, reprocessing factible"
2,Delta,Baja-Media (micro-batch),Media (un storage),"Lakehouse, unificación batch/stream"
3,Data Mesh,Variable (por dominio),Alta (organizacional),Grandes org con múltiples dominios


In [8]:
# 📊 RESUMEN EJECUTIVO: Decisión de Arquitectura
import pandas as pd
import matplotlib.pyplot as plt

print("=" * 80)
print("🎯 GUÍA DE DECISIÓN: ¿QUÉ ARQUITECTURA ELEGIR?")
print("=" * 80)

# ============= MATRIZ DE DECISIÓN =============
print("\n📋 MATRIZ DE DECISIÓN POR CASO DE USO\n")

decision_matrix = pd.DataFrame([
    {
        'Escenario': 'Legacy con batch + necesidad streaming',
        'Mejor Opción': 'Lambda',
        'Razón': 'Evolución gradual sin reescribir',
        'Complejidad': '⚠️ Alta',
        'Costo': '💰💰💰'
    },
    {
        'Escenario': 'Todo es real-time, histórico < 1 año',
        'Mejor Opción': 'Kappa',
        'Razón': 'Simplicidad operacional',
        'Complejidad': '✅ Media',
        'Costo': '💰💰'
    },
    {
        'Escenario': 'Data warehouse + lake unificado',
        'Mejor Opción': 'Delta',
        'Razón': 'ACID + time travel + bajo costo',
        'Complejidad': '✅ Media',
        'Costo': '💰'
    },
    {
        'Escenario': 'Org grande (500+ empleados), múltiples dominios',
        'Mejor Opción': 'Data Mesh',
        'Razón': 'Escala organizacional',
        'Complejidad': '⚠️⚠️ Muy Alta',
        'Costo': '💰💰💰💰'
    },
    {
        'Escenario': 'Startup MVP, equipo pequeño',
        'Mejor Opción': 'Delta (simple)',
        'Razón': 'Mínima complejidad operacional',
        'Complejidad': '✅ Baja',
        'Costo': '💰'
    },
    {
        'Escenario': 'Fintech con latencia <100ms',
        'Mejor Opción': 'Kappa',
        'Razón': 'Streaming puro con estado',
        'Complejidad': '⚠️ Alta',
        'Costo': '💰💰💰'
    },
    {
        'Escenario': 'Compliance GDPR + auditabilidad',
        'Mejor Opción': 'Delta',
        'Razón': 'Time travel + ACID transactions',
        'Complejidad': '✅ Media',
        'Costo': '💰'
    }
])

print(decision_matrix.to_string(index=False))

# ============= COMPARATIVA CUANTITATIVA =============
print("\n" + "=" * 80)
print("📊 COMPARATIVA CUANTITATIVA (basada en simulaciones)")
print("=" * 80)

results_summary = pd.DataFrame([
    {
        'Arquitectura': 'Lambda',
        'Eventos Procesados': '10,000',
        'Latencia Batch': '9ms',
        'Latencia Speed': '3ms',
        'Merge Overhead': 'Sí',
        'Complejidad Código': '2 pipelines',
        'Storage GB': '0.5 + Kafka'
    },
    {
        'Arquitectura': 'Kappa',
        'Eventos Procesados': '15,000',
        'Latencia Batch': 'N/A (replay)',
        'Latencia Speed': '0.13ms/evento',
        'Merge Overhead': 'No',
        'Complejidad Código': '1 pipeline',
        'Storage GB': '7.32 (Kafka 30d)'
    },
    {
        'Arquitectura': 'Delta',
        'Eventos Procesados': '20,300',
        'Latencia Batch': '80ms',
        'Latencia Speed': '5ms (micro-batch)',
        'Merge Overhead': 'No',
        'Complejidad Código': '1 pipeline',
        'Storage GB': '0.28 (Parquet)'
    },
    {
        'Arquitectura': 'Data Mesh',
        'Eventos Procesados': '14,300 (3 dominios)',
        'Latencia Batch': 'Variable',
        'Latencia Speed': 'Variable',
        'Merge Overhead': 'Cross-domain joins',
        'Complejidad Código': 'N pipelines',
        'Storage GB': '0.5 (distributed)'
    }
])

print(results_summary.to_string(index=False))

# ============= CRITERIOS DE SELECCIÓN =============
print("\n" + "=" * 80)
print("🔍 CRITERIOS CLAVE PARA LA DECISIÓN")
print("=" * 80)

criteria = {
    '1. Latencia Requerida': {
        '< 100ms': '→ Kappa (streaming puro)',
        '< 1 segundo': '→ Kappa o Lambda speed layer',
        '< 30 segundos': '→ Delta (micro-batch)',
        '> 1 minuto': '→ Lambda batch o Delta batch'
    },
    '2. Tamaño Histórico': {
        '< 6 meses': '→ Kappa (Kafka retention factible)',
        '6 meses - 2 años': '→ Kappa o Delta (depende de costo)',
        '> 2 años': '→ Delta o Lambda (S3/ADLS barato)'
    },
    '3. Madurez Organizacional': {
        'Startup (< 50)': '→ Delta (simplicidad)',
        'Mediana (50-500)': '→ Lambda o Delta (centralizado)',
        'Enterprise (> 500)': '→ Data Mesh (descentralización)'
    },
    '4. Requisitos de Compliance': {
        'Auditabilidad': '→ Delta (time travel)',
        'GDPR right to be forgotten': '→ Delta (ACID deletes)',
        'Inmutabilidad': '→ Kappa (log inmutable)',
        'Retención 5-10 años': '→ Delta o Lambda'
    }
}

for category, options in criteria.items():
    print(f"\n{category}:")
    for condition, recommendation in options.items():
        print(f"  • {condition}: {recommendation}")

# ============= ANTI-PATTERNS =============
print("\n" + "=" * 80)
print("❌ ANTI-PATTERNS (Qué NO hacer)")
print("=" * 80)

anti_patterns = [
    '❌ Usar Lambda si estás empezando de cero (mejor Delta)',
    '❌ Usar Kappa con históricos de 5+ años (costo prohibitivo)',
    '❌ Implementar Data Mesh con equipo < 50 personas (overhead)',
    '❌ Usar Delta para latencia < 1s crítica (micro-batch tiene overhead)',
    '❌ Migrar de Lambda a Kappa sin plan de reprocessing',
    '❌ Adoptar Data Mesh sin plataforma self-service',
    '❌ Usar streaming para ETL batch simples (overkill)'
]

for pattern in anti_patterns:
    print(f"  {pattern}")

# ============= RESUMEN FINAL =============
print("\n" + "=" * 80)
print("✅ RESUMEN EJECUTIVO")
print("=" * 80)

print("""
🏗️ LAMBDA: 
   ✅ Cuando: Legacy batch + necesitas agregar streaming
   ⚠️ Cuidado: 2x complejidad, doble código

♾️ KAPPA:
   ✅ Cuando: Todo es streaming, histórico < 1 año
   ⚠️ Cuidado: Kafka storage caro, estado complejo

🏛️ DELTA:
   ✅ Cuando: Lakehouse unificado, GDPR, analytics + streaming
   ⚠️ Cuidado: Latencia > 1s (micro-batch overhead)

🌐 DATA MESH:
   ✅ Cuando: Enterprise > 500 personas, múltiples dominios
   ⚠️ Cuidado: Requiere madurez DevOps/SRE senior

💡 REGLA DE ORO:
   • < 50 personas → Delta (simplicidad)
   • 50-500 personas → Delta o Kappa (según latencia)
   • > 500 personas → Data Mesh sobre Delta/Kappa
   • Legacy modernizando → Lambda → gradual a Delta
""")

print("=" * 80)
print("🎓 Notebook completado - Arquitecturas Modernas dominadas!")
print("=" * 80)

🎯 GUÍA DE DECISIÓN: ¿QUÉ ARQUITECTURA ELEGIR?

📋 MATRIZ DE DECISIÓN POR CASO DE USO

                                      Escenario   Mejor Opción                            Razón   Complejidad Costo
         Legacy con batch + necesidad streaming         Lambda Evolución gradual sin reescribir       ⚠️ Alta   💰💰💰
           Todo es real-time, histórico < 1 año          Kappa          Simplicidad operacional       ✅ Media    💰💰
                Data warehouse + lake unificado          Delta  ACID + time travel + bajo costo       ✅ Media     💰
Org grande (500+ empleados), múltiples dominios      Data Mesh            Escala organizacional ⚠️⚠️ Muy Alta  💰💰💰💰
                    Startup MVP, equipo pequeño Delta (simple)   Mínima complejidad operacional        ✅ Baja     💰
                    Fintech con latencia <100ms          Kappa        Streaming puro con estado       ⚠️ Alta   💰💰💰
                Compliance GDPR + auditabilidad          Delta  Time travel + ACID transactions       ✅

## 6. Ejercicio de diseño

Diseña la arquitectura de datos para un e-commerce con:
- Dominio Ventas: transacciones en tiempo real (Kafka).
- Dominio Logística: batch nocturno (inventario/envíos).
- Dominio Analítica: dashboards con latencia < 5 min.
- Requisitos: auditabilidad, GDPR, linaje, costos optimizados.

Responde: ¿Lambda, Kappa, Delta o híbrida? ¿Data Mesh aplicable? Justifica con trade-offs.

---

## 🧭 Navegación

**← Anterior:** [Apache Spark Streaming: Procesamiento en Tiempo Real](03_spark_streaming.ipynb)

**Siguiente →:** [🤖 ML Pipelines y Feature Stores →](05_ml_pipelines_feature_stores.ipynb)

**📚 Índice de Nivel Senior:**
- [🏛️ Senior - 01. Data Governance y Calidad de Datos](01_data_governance_calidad.ipynb)
- [🏗️ Data Lakehouse con Parquet, Delta Lake e Iceberg (conceptos y práctica ligera)](02_lakehouse_delta_iceberg.ipynb)
- [Apache Spark Streaming: Procesamiento en Tiempo Real](03_spark_streaming.ipynb)
- [🏛️ Arquitecturas Modernas de Datos: Lambda, Kappa, Delta y Data Mesh](04_arquitecturas_modernas.ipynb) ← 🔵 Estás aquí
- [🤖 ML Pipelines y Feature Stores](05_ml_pipelines_feature_stores.ipynb)
- [💰 Cost Optimization y FinOps en la Nube](06_cost_optimization_finops.ipynb)
- [🔐 Seguridad, Compliance y Auditoría de Datos](07_seguridad_compliance.ipynb)
- [📊 Observabilidad y Linaje de Datos](08_observabilidad_linaje.ipynb)
- [🏆 Proyecto Integrador Senior 1: Plataforma de Datos Completa](09_proyecto_integrador_1.ipynb)
- [🌐 Proyecto Integrador Senior 2: Data Mesh Multi-Dominio con Feature Store](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
